In [2]:
import tkinter as tk
from tkinter import messagebox

# Onsight Competition Score Application

## Goals
- ☐ Add a menu bar (tk.Menu)
- ☐ Consider allowing for switching between data entry types, or selecting which method you'd like to use?
- ☑ Add toggle for showing score from each boulder in scoring methods where it applies. (currently in ScoringOlympic)
- ☐ Add a changing title to the window based on which frame is active.

### IFSC
- ☑ Identify scoring rules
- ☑ Create logic in a small scale (not using tkinter)
- ☑ Create and develop GUI class for data entry/leaderboard
- ☑ Pair logic to GUI, working with optimal data entry
- ☑ Add complete functionality to buttons 
- ☑ Implement rules for data entry - Function validate_scores
- ☑ Set up error messages
- ☐ Final testing
- ☐ Push to Main



### USAC
- ☑ Identify scoring rules
- ☑ Create logic in a small scale (not using tkinter)
- ☑ Create GUI class for data entry/leaderboard
- ☑ Pair logic to GUI, working with optimal data entry - refactor with new format from IFSC and Olympic
- ☐ Add functionality to buttons
- ☐ Implement rules for data entry
- ☐ Set up error messages
- ☐ Final testing
- ☐ Push to Main



### Olympic
- ☑ Identify scoring rules
- ☑ Create logic in a small scale (not using tkinter)
- ☑ Create GUI class for data entry/leaderboard 
- ☑ Pair logic to GUI, working with optimal data entry
- ☑ Add functionality to buttons
- ☑ Implement rules for data entry - validate_scores()
- ☑ Set up error messages
- ☐ Final testing
- ☐ Push to Main

<u>Current problems to fix:</u>
- ☑ Need functionality to some buttons like clear_leaderboard.
- ☑ Fix the edit_climber function. Currently does not update the leaderboard, maybe doesn't even make it through the loop (debug prints).
- ☑ Validate scores that works with the RadioButton setup.
- ☑ Add an on-off switch for showing each boulder's score below the score sum. (Total Score: 25, B1: 5, B2: 15, B3: 2.5, B4: 2.5)


In [ ]:
import tkinter as tk
from tkinter import ttk, messagebox
# Creating the starting window to select the different scoring types.
# Creating (currently 4) scoring frames, when one is selected, use .tkraise() to bring selected scoring method to the forefront of the UI
class App(tk.Tk):
    """Base level window that holds the other frames"""
    def __init__(self):
        super().__init__()
        self.title("Scoring App")

        container = tk.Frame(self)
        container.pack(fill= "both", expand= True)

        self.frames = {}

        for F in (StartingWindow, ScoringUSAC, ScoringOlympic, ScoringIFSC25):
            frame = F(container, self)
            self.frames[F.__name__] = frame
            frame.grid(row=0, column=0, sticky="nsew")

        self.show_frame("StartingWindow")

    def show_frame(self, name):
        self.frames[name].tkraise()

class StartingWindow(tk.Frame):
    """First window where the type of scoring is selected"""
    def __init__(self, parent, controller):
        super().__init__(parent)
        controller.title("Home Window")

        # Window
        self.starting_frame = ttk.Frame(self)
        self.starting_frame.grid(column= 0, row= 0, padx= 20, pady= 20, sticky= 'nsew')
        self.grid_columnconfigure(0, weight= 1)
        self.grid_rowconfigure(0, weight= 1)

        # Text
        self.choice_text = ttk.Label(self.starting_frame, text= "Please select your Scoring Style!")
        self.choice_text.grid(column= 0, row= 0, padx= 20, pady= 20, columnspan= 2)
        self.starting_frame.columnconfigure((0, 1), weight= 1)
        self.starting_frame.rowconfigure(1, weight= 1)

        # Buttons
        self.button_USAC = ttk.Button(self.starting_frame, text= "USAC Zone Style", command= lambda: controller.show_frame("ScoringUSAC"))
        self.button_USAC.grid(column= 0, row= 1, ipadx= 20, ipady= 20, sticky= 'ew')

        self.button_Olympic = ttk.Button(self.starting_frame, text= "Olympic Style", command= lambda: controller.show_frame("ScoringOlympic"))
        self.button_Olympic.grid(column= 1, row= 1, ipadx= 20, ipady= 20, sticky= 'ew')

        self.button_IFSC = ttk.Button(self.starting_frame, text= "IFSC 2025 Style", command= lambda: controller.show_frame("ScoringIFSC25"))
        self.button_IFSC.grid(column= 0, row= 2, ipadx= 20, ipady= 20, sticky= 'ew')

        self.button_OldUSAC = ttk.Button(self.starting_frame, text= "Old USAC Style", command= lambda: messagebox.showinfo(message="Not available yet"))
        self.button_OldUSAC.grid(column= 1, row= 2, ipadx= 20, ipady= 20, sticky= 'ew')


class ScoringUSAC(tk.Frame):
    """The classic scoring window layout for USAC and IFSC events"""
    class Boulder:
        def __init__(self, attempts, level):
            self.attempts = attempts
            self.level = level if level else "0"
            self.top = 0
            self.zone = 0
            self.lowzone = 0
            self.top_attempts = 0
            self.zone_attempts = 0
            self.lowzone_attempts = 0
            self.score = self.calculate_score()
        
        def calculate_score(self):
            if self.level == "T":
                self.top = self.zone = self.lowzone = 1
                self.top_attempts = self.zone_attempts = self.lowzone_attempts = self.attempts
            elif self.level == "Z":
                self.zone = self.lowzone = 1
                self.zone_attempts = self.lowzone_attempts = self.attempts
            elif self.level == "LZ":
                self.lowzone = 1
                self.lowzone_attempts = self.attempts
            else:
                return # Nothing is scored

    class Climber:
        def __init__(self, name):
            self.name = name
            self.boulder_list = []
            self.rank = None

        def add_boulder(self, attempts, level):
            """Adds level and attempts of the boulder to the boulder_list"""
            self.boulder_list.append(ScoringUSAC.Boulder(attempts, level))

        def total_score(self):
            total_tops = sum(boulder.top for boulder in self.boulder_list)
            total_zones = sum(boulder.zone for boulder in self.boulder_list)
            total_lowzones = sum(boulder.lowzone for boulder in self.boulder_list)
            total_top_attempts = sum(boulder.top_attempts for boulder in self.boulder_list)
            total_zone_attempts = sum(boulder.zone_attempts for boulder in self.boulder_list)
            total_lowzone_attempts = sum(boulder.lowzone_attempts for boulder in self.boulder_list)
            
            return (f"Levels= T:{total_tops}, Z:{total_zones}, LZ:{total_lowzones}\nAttempts= T:{total_top_attempts}, Z:{total_zone_attempts}, LZ:{total_lowzone_attempts} ")

        def delete(self, leaderboard):
            """Deletes the climber from the leaderboard."""
            leaderboard.climbers.remove(self)

        def __str__(self):
            return f"Name: {self.name}\nLevels= T:{self.total_tops}, Z:{self.total_zones}, LZ:{self.total_lowzones}\nAttempts= T:{self.total_top_attempts}, Z:{self.total_zone_attempts}, LZ:{self.total_lowzone_attempts}"
    
    class Leaderboard:
        def __init__(self):
            self.climbers = []
            self.show_score_breakdown = False

        def add_climber(self, climber):
            self.climbers.append(climber)

        def score_breakdown(self):
            """Toggles score breakdowns"""
            self.show_score_breakdown = not self.show_score_breakdown

        def rank_climbers(self):
            """Ranks climbers by the specified key"""
            self.climbers.sort(key= lambda c: (
                -sum(boulder.top for boulder in c.boulder_list), #decending, c for climber and b for boulder
                -sum(boulder.zone for boulder in c.boulder_list), #decending
                -sum(boulder.lowzone for boulder in c.boulder_list), #decending
                sum(boulder.top_attempts for boulder in c.boulder_list), #ascending
                sum(boulder.zone_attempts for boulder in c.boulder_list), #ascending
                sum(boulder.lowzone_attempts for boulder in c.boulder_list) #ascending
            ))
            # Not dealing with tie's in this format
            for i, climber in enumerate(self.climbers, start= 1):
                climber.rank = i

        def __str__(self):
            result = []
            for climber in self.climbers:
                breakdown = ""
                if self.show_score_breakdown:
                    # totals for each scoring type
                    total_tops = sum(boulder.top for boulder in climber.boulder_list)
                    total_zones = sum(boulder.zone for boulder in climber.boulder_list)
                    total_lowzones = sum(boulder.lowzone for boulder in climber.boulder_list)
                    total_top_attempts = sum(boulder.top_attempts for boulder in climber.boulder_list)
                    total_zone_attempts = sum(boulder.zone_attempts for boulder in climber.boulder_list)
                    total_lowzone_attempts = sum(boulder.lowzone_attempts for boulder in climber.boulder_list)
                    breakdown = (
                        f"    {total_tops} Tops, {total_top_attempts} attempts.\n"
                        f"    {total_zones} Zones, {total_zone_attempts} attempts.\n"
                        f"    {total_lowzones} LowZones, {total_lowzone_attempts} attempts."
                    )
                result.append(f"{climber.rank}. {climber.name}\n{breakdown}")
            return "\n".join(result)

    def __init__(self, parent, controller):
        super().__init__(parent)

        # Main frame
        self.main_frame = ttk.Frame(self)
        self.main_frame.grid(column= 0, row= 0, padx= 20, pady= 20, sticky= "nsew")
        self.main_frame.rowconfigure(0, weight= 1)
        self.main_frame.columnconfigure(0, weight= 1)
        self.main_frame.columnconfigure(1, weight= 1)
        self.leaderboard = self.Leaderboard()

        self.max_boulders = 11
        self.boulder_widgets = []
        self.scoretype_vars = []

        self.create_left()
        self.create_right()

        backbutton = ttk.Button(self, text= "Back", command= lambda: self.backbutton_popup(controller))
        backbutton.grid(column= 0, row= 1, columnspan= 2, sticky= 's')

    def backbutton_popup(self, controller):
        if messagebox.askyesno("Warning!", "Are you sure you want to go back?\nData isn't shared between scoring styles."):
            controller.show_frame("StartingWindow")
        else:
            pass 

    def create_left(self):
        """Left side creation where labels and entry fields exist to gather information from the user."""
        self.left_frame = ttk.Frame(self.main_frame)
        self.left_frame.grid(row=0, column=0, padx=10, pady= 5, sticky= 'nw')

        # Resizing parameters
        for i in range(0, 3):
            self.left_frame.columnconfigure(i, weight= 1)

        for i in range(0, 20):
            self.left_frame.rowconfigure(i, weight= 1)

        # Name Label and Entry
        self.name_label = ttk.Label(self.left_frame, text= "Climber Name : ")
        self.name_label.grid(column= 0, row= 1, padx=5, pady=5, sticky="e")
        self.name_entry = ttk.Entry(self.left_frame, justify= tk.CENTER)
        self.name_entry.grid(column= 1, row= 1, padx=5, pady=5, sticky="w")

        # Dropdown Menu for Number of Boulders
        self.boulder_count_var = tk.StringVar()
        self.boulder_count_select = ttk.Combobox(
            self.left_frame, textvariable= self.boulder_count_var, state= "readonly",
            values= [str(i) for i in range(1, self.max_boulders)], takefocus= 0
        )
        self.boulder_count_select.set("Number of Boulders")
        self.boulder_count_select.bind("<<ComboboxSelected>>", self.update_boulder_fields)
        self.boulder_count_select.grid(column= 1, row= 0, columnspan= 2, pady= 10)

        # Label, entry, and radiobutton fields
        radio_options = ["T", "Z", "LZ", "None"]
        for i in range(self.max_boulders):
            label = ttk.Label(self.left_frame, text=f"Boulder {i + 1} Attempts:")
            entry = ttk.Entry(self.left_frame, width=20, justify=tk.CENTER)

            score_var = tk.StringVar()
            self.scoretype_vars.append(score_var)

            # radiobutton frame
            rb_frame = ttk.Frame(self.left_frame)
            for optn in radio_options:
                rb = ttk.Radiobutton(
                    rb_frame,
                    text= optn,
                    variable= score_var,
                    value= optn,
                    takefocus= False
                )
                rb.pack(side= "left", padx= 2)

            # Putting label, entry, and radiobuttons all in 1 row each
            label.grid(column= 0, row= i + 2, padx= 5, pady= 5)
            entry.grid(column= 1, row= i + 2, padx= 5, pady= 5, sticky= "w")
            rb_frame.grid(column= 2, row= i + 2, padx= 5, pady= 5, sticky= "w")

            # Hide until called
            label.grid_remove()
            entry.grid_remove()
            rb_frame.grid_remove()

            self.boulder_widgets.append((label, entry, rb_frame))

        # Grey text for entry instructions
        self.instruction_label = ttk.Label(
            self.left_frame, 
            foreground= "gray",  
            text= "Enter attempts to the highest scored level. \nNext, select the highest scored level\n"
        )
        self.instruction_label.grid(column= 0, columnspan= 2, row= self.max_boulders + 1)
        
        # Leftside Buttons
        self.add_climber_button = ttk.Button(self.left_frame, text= "Add Climber", command= lambda: self.add_climber())
        self.add_climber_button.grid(row= self.max_boulders + 2, column= 0, columnspan= 2, pady= 10)

        self.edit_climber_button = ttk.Button(self.left_frame, text= "Edit Climber")
        self.edit_climber_button.grid(column= 0, row= self.max_boulders + 3, sticky= 'w', padx=5, pady=5)

        self.remove_climber_button = ttk.Button(self.left_frame, text= "Remove Climber", command= lambda: self.remove_climber())
        self.remove_climber_button.grid(column= 1, row= self.max_boulders + 3, sticky= 'e', padx=5, pady=5)

    def create_right(self):
        """Create the right frame for the leaderboard display and associated buttons"""
        self.right_frame = ttk.Frame(self.main_frame)
        self.right_frame.grid(row=0, column=1, padx=10, sticky= 'nsew')

        #letting right frame expand to fill the space
        for i in range(0, 3):
            self.right_frame.columnconfigure(i, weight= 1)

        for i in range(0, 12):
            self.right_frame.rowconfigure(i, weight= 1)

        # Leaderboard Display
        self.leaderboard_text = tk.Text(self.right_frame, height=20, width=30)
        self.leaderboard_text.grid(row=0, column=0, columnspan= 2, pady=10, sticky= "nsew")
        self.disable_edits()

        # Rightside Buttons
        self.leaderboard_buttons = ttk.Frame(self.right_frame)
        self.leaderboard_buttons.grid(row=1, column=0, columnspan=2, pady=10)
        ttk.Button(self.leaderboard_buttons, text="Clear Leaderboard").grid(row= 0, column= 0, columnspan= 2, padx= 10, pady= 10) # command=self.clear_leaderboard_ask

        breakdown_toggle_var = tk.BooleanVar(value= False)
        score_breakdown_checkbutton = ttk.Checkbutton(
            self.leaderboard_buttons, 
            text= "Show breakdown of Scores", 
            variable= breakdown_toggle_var, 
            command= lambda: self.toggle_score_breakdown(breakdown_toggle_var.get())
            )
        score_breakdown_checkbutton.grid(row= 1, column= 0, padx= 10, pady= 10)

    # Data Entry and creation functions
    def update_boulder_fields(self, event= None):
        """Show/Hide boulder label/entry fields based on selected number of boulders."""
        selected_count = int(self.boulder_count_var.get())
        # Show the required number of boulder fields
        for i, (label, entry, radio) in enumerate(self.boulder_widgets):
            if i < selected_count:
                label.grid()
                entry.grid()
                radio.grid()
            else:
                label.grid_remove()
                entry.grid_remove()
                radio.grid_remove()
        self.left_frame.update_idletasks()   

    def clear_entries(self):
        selected_boulders = int(self.boulder_count_var.get())
        for i in range(selected_boulders):
            self.boulder_widgets[i][1].delete(0, tk.END)
            try:
                self.scoretype_vars[i].set('')  #set each button to '' in every iteration
            except Exception as e:
                print(f"{e}")
        self.name_entry.delete(0, tk.END)

    # Manipulating Climbers and Leaderboards classes
    def add_climber(self):
        self.enable_edits()
        try:
            climber_name = self.name_entry.get().strip()
            if not climber_name:
                raise ValueError(f"Climber name cannot be empty")
            for climber in self.leaderboard.climbers:
                if climber.name == climber_name:
                    raise ValueError(f"{climber_name} already exists in the leaderboard.\nPlease add another identifier")
            try:
                selected_boulders = int(self.boulder_count_var.get())
            except:
                raise ValueError(f"You must select a valid number of boulders to score.")
            #self.validate_scores() - once function has been added. Making sure attempts is an integer, and some radiobutton has been selected

            boulder_list = []
            for i in range(selected_boulders):
                attempts_entry = self.boulder_widgets[i][1]
                attempts_str = attempts_entry.get()
                level_str = self.scoretype_vars[i].get()

                if attempts_str in ["0", ""]:
                    attempts = 0
                    level = "0"
                else:
                    attempts = int(attempts_str)
                    level = level_str if level_str else "0"
                boulder_list.append(self.Boulder(attempts, level))
            
            climber = self.Climber(climber_name)
            climber.boulder_list = boulder_list
            self.leaderboard.add_climber(climber)

            messagebox.showinfo("Success", f"Scores for {climber_name} added!")
            self.update_leaderboard()
        except Exception as e:
            messagebox.showerror("Error", f"{e}")

    def edit_climber(self):
        pass

    def remove_climber(self):
        try:
            name = self.name_entry.get()
            if not name:
                raise ValueError(f"Climber name cannot be empyt")
            found = False
            for climber in self.leaderboard.climbers[:]:
                if climber.name == name:
                    self.leaderboard.climbers.remove(climber)
                    found = True
            if found:
                messagebox.showinfo("Success", f"'{name}' has been removed." )
                self.update_leaderboard()
            else:
                raise ValueError(f"Climber {name} not found in the leaderboard.")
        except Exception as e:
            messagebox.showerror("Error", f"{e}")

    # Edits to the Leaderboard
    def update_leaderboard(self):
        """Clears the leaderboard, re-ranks climbers, and re-prints the leaderboard."""
        self.enable_edits()
        self.leaderboard.rank_climbers()
        self.leaderboard_text.delete(1.0, tk.END)
        self.leaderboard_text.insert(tk.END, str(self.leaderboard))
        self.clear_entries()
        self.disable_edits()

    def validate_scores(self):
        pass

    def toggle_score_breakdown(self, show_breakdown):
        self.leaderboard.show_score_breakdown = show_breakdown
        self.update_leaderboard()

    def enable_edits(self):
        """Enables editing of the leaderboard."""
        self.leaderboard_text.config(state= tk.NORMAL)

    def disable_edits(self):
        """Disables editing of the leaderboard."""
        self.leaderboard_text.config(state= tk.DISABLED)


class ScoringOlympic(tk.Frame):
    """Olympic Style scoring that is more points focused rather than zone focused"""
    class Boulder:
        """Represents a single boulder and the attempts to a scoring level, as well as the calculated score.
            * attempts: Number of attempts made to highest scored space.
            * level: Highest achieved scoring space (T for top, Z for zone, 0 for no score)"""

        def __init__(self, attempts, level):
            self.attempts = attempts
            self.level = level if level else "0"
            self.score = self.calculate_score()

        def calculate_score(self):
            """Calculates the score for the boulder based on the highest scored level and attempts to that level.
                * return: calculated score."""
            if self.level == "25":
                return round(25 - ((self.attempts - 1) * 0.1), 5) # using attempts -1 because the score technically only subtracts .1 per "failed" attempt
            elif self.level == "10":
                return round(10 - ((self.attempts - 1) * 0.1), 5)
            elif self.level == "5":
                return round(5 - ((self.attempts - 1) * 0.1), 5)
            elif self.level == "" or self.level == "0":
                return 0

    class Climber:
        def __init__(self, name):
            self.name = name
            self.boulder_list = []
            self.rank = None    # Adding rankings rather than just listing order

        def add_boulder(self, attempts, level):
            """Adds the score of a boulder to the climber's scorecard."""
            self.boulder_list.append(ScoringOlympic.Boulder(attempts, level))

        def total_score(self):
            """Calculates the total score of the climber. Used for ranking on the leaderboard."""
            total = round(sum(boulder.score for boulder in self.boulder_list), 5)
            return total
        
        def delete(self, leaderboard):
            """Deletes the climber from the leaderboard."""
            leaderboard.climbers.remove(self)
        
        def __str__(self):
            return f"{self.name}: {self.total_score():.1f} total points."
        
    class Leaderboard:
        """Leaderboard class that will visualize ranking of climbers based on their scores."""
        def __init__(self):
            self.climbers = []
            self.show_score_breakdown = False #Default state

        def add_climber(self, climber):
            """Adds a climber to the leaderboard."""
            self.climbers.append(climber)
            
        def rank_climbers(self):
            """Sorts climbers by total score, in descending order."""
            self.climbers.sort(key= lambda climber: climber.total_score(), reverse= True)
            tie_count = 0
            previous_score = None
            current_rank = 1
            tolerance = 1e-5

            for climber in self.climbers:       # New version no longer needs to reference i or enumerate over self.climbers since variables are now used.
                current_score = round(climber.total_score(), 5)

                if previous_score is not None and abs(current_score - previous_score) < tolerance:
                    climber.rank = current_rank
                    tie_count += 1
                else:
                    current_rank = current_rank + tie_count
                    climber.rank = current_rank
                    tie_count = 1
                previous_score = current_score

        def show_score_breakdown(self):
            """Toggle score breakdown"""
            self.show_score_breakdown = not self.show_score_breakdown

        def __str__(self):
            """Generate a string representation of the leaderboard."""
            result = []
            for climber in self.climbers:
                breakdown = ""
                if self.show_score_breakdown:
                    breakdown = ", ".join(
                        [f"B{i + 1}: {boulder.score:.1f}" for i, boulder in enumerate(climber.boulder_list)]
                    )
                    breakdown = f" ({breakdown})"
                result.append(f"{climber.rank}. {climber.name}: {climber.total_score():.1f} total points\n{breakdown}")
            return "\n".join(result)


    def __init__(self, parent, controller):
        super().__init__(parent)
                # Main Frame
        self.main_frame = ttk.Frame(self)
        self.main_frame.grid(column= 0, row= 0, padx= 20, pady= 20, sticky= "nsew")
        self.main_frame.rowconfigure(0, weight= 1)
        self.main_frame.columnconfigure(0, weight= 1)
        self.main_frame.columnconfigure(1, weight= 1)
        self.leaderboard = self.Leaderboard()

        self.max_boulders = 11
        self.boulder_widgets = []
        self.scoretype_vars = []

        self.create_left_frame()
        self.create_right_frame()

        backbutton = ttk.Button(self, text= "Back", command= lambda: self.backbutton_popup(controller))
        backbutton.grid(column= 0, row= 1, columnspan= 2, sticky= "s")

    def backbutton_popup(self, controller):
        """Reminds the user that data isn't shared, then brings the StartingWindow frame to the top."""
        if messagebox.askyesno("Warning!", "Are you sure you want to go back?\nData isn't shared between scoring styles."):
            controller.show_frame("StartingWindow")
        else:
            pass

    def create_left_frame(self):
        """Create left frame where labels and entry fields exist to gather information from the user."""
        self.left_frame = ttk.Frame(self.main_frame)
        self.left_frame.grid(row=0, column=0, padx= 10, pady= 5, sticky= 'nw')

        # Resizing parameters
        for i in range(0, 3):
            self.left_frame.columnconfigure(i, weight= 1)

        for i in range(0, 20):
            self.left_frame.rowconfigure(i, weight= 1)

        # Name Label and Entry
        self.name_label = ttk.Label(self.left_frame, text= "Climber Name : ")
        self.name_label.grid(column= 0, row= 1, padx=5, pady=5, sticky="e")
        self.name_entry = ttk.Entry(self.left_frame, justify= tk.CENTER)
        self.name_entry.grid(column= 1, row= 1, padx=5, pady=5, sticky="w")

        # Dropdown Menu for Number of Boulders
        self.boulder_count_var = tk.StringVar()
        self.boulder_count_select = ttk.Combobox(
            self.left_frame, textvariable= self.boulder_count_var, state= "readonly",
            values= [str(i) for i in range(1, self.max_boulders)], takefocus= 0
        )
        self.boulder_count_select.set("Number of Boulders")
        self.boulder_count_select.bind("<<ComboboxSelected>>", self.update_boulder_fields)
        print(f"Updating Boulder Fields. Max boulders: {self.max_boulders}")
        self.boulder_count_select.grid(column= 1, row= 0, columnspan= 2, pady= 10)

        score_radio_options = ['25', '10', '5', '0']
        # Creating all widgets as if max number of boulders is selected
        for i in range(self.max_boulders):
            label = ttk.Label(self.left_frame, text=f"Boulder {i + 1} Attempts:")
            entry = ttk.Entry(self.left_frame, width=20, justify=tk.CENTER)

            score_var = tk.StringVar()
            self.scoretype_vars.append(score_var)

            # radiobutton frame
            rb_frame = ttk.Frame(self.left_frame)
            for optn in score_radio_options:
                rb = ttk.Radiobutton(
                    rb_frame,
                    text= optn,
                    variable= score_var,
                    value= optn,
                    takefocus= False
                )
                rb.pack(side= "left", padx= 2)
            # Putting label, entry, and radiobuttons all in 1 row each
            label.grid(column= 0, row= i + 2, padx= 5, pady= 5)
            entry.grid(column= 1, row= i + 2, padx= 5, pady= 5, sticky= "w")
            rb_frame.grid(column= 2, row= i + 2, padx= 5, pady= 5, sticky= "w")

            # Hide until called
            label.grid_remove()
            entry.grid_remove()
            rb_frame.grid_remove()

            self.boulder_widgets.append((label, entry, rb_frame))

        # Grey text for entry instructions
        self.instruction_label = ttk.Label(
            self.left_frame, 
            foreground= "gray",  
            text= "Enter attempts to the highest scored level. \nIn the second box, select the highest scored level"
        )
        self.instruction_label.grid(column= 0, columnspan= 2, row= self.max_boulders + 1)
        
        # Left Frame buttons
        self.add_climber_button = ttk.Button(self.left_frame, text= "Add Climber", command= lambda: self.add_climber())
        self.add_climber_button.grid(column= 0, columnspan= 2, row= self.max_boulders + 2, padx=5, pady=5)

        self.edit_climber_button = ttk.Button(self.left_frame, text= "Edit Climber", command= lambda: self.edit_climber())
        self.edit_climber_button.grid(column= 0, row= self.max_boulders + 3, sticky= 'w', padx=5, pady=5)

        self.remove_climber_button = ttk.Button(self.left_frame, text= "Remove Climber", command= lambda: self.remove_climber())
        self.remove_climber_button.grid(column= 1, row= self.max_boulders + 3, sticky= 'e', padx=5, pady=5)

    def create_right_frame(self):
        """Create the right frame for the leaderboard display and associated buttons.
        Should be the same for all scoring types."""
        self.right_frame = ttk.Frame(self.main_frame)
        self.right_frame.grid(row= 0, column= 1, padx= 10, sticky= 'nsew')

        # Resizing parameters
        for i in range(0, 3):
            self.right_frame.columnconfigure(i, weight= 1)

        for i in range(0, 12):
            self.right_frame.rowconfigure(i, weight= 1)

        # Leaderboard Display
        self.leaderboard_text = tk.Text(self.right_frame, height= 20, width= 40)
        self.leaderboard_text.grid(row=0, column=0, columnspan= 2, pady= 10, sticky= "nse")
        self.disable_edits()

        # Rightside Buttons
        self.leaderboard_buttons = ttk.Frame(self.right_frame)
        self.leaderboard_buttons.grid(row=1, column=0, columnspan=2, pady=10)
        ttk.Button(self.leaderboard_buttons, text="Clear Leaderboard", command= lambda: self.clear_leaderboard_ask()).grid(row= 0, column= 0, columnspan= 2, padx= 10, pady= 10)#.pack(side= "left", padx= 10, pady= 10)

        breakdown_toggle_var = tk.BooleanVar(value= False)
        score_breakdown_checkbutton = ttk.Checkbutton(
            self.leaderboard_buttons, 
            text= "Show breakdown of Scores", 
            variable= breakdown_toggle_var, 
            command= lambda: self.toggle_score_breakdown(breakdown_toggle_var.get())
            )
        score_breakdown_checkbutton.grid(row= 1, column= 0, padx= 10, pady= 10)

    def update_boulder_fields(self, event= None):
        """Show/Hide boulder label/entry fields based on selected number of boulders."""
        selected_count = int(self.boulder_count_var.get())
        # Show the required number of boulder fields
        for i, (label, entry, combobox) in enumerate(self.boulder_widgets):
            if i < selected_count:
                label.grid()
                entry.grid()
                combobox.grid()
            else:
                label.grid_remove()
                entry.grid_remove()
                combobox.grid_remove()
        self.left_frame.update_idletasks()

    def clear_entries(self):
        """Clears the entry fields, saves the number of boulders selected."""
        selected_boulders = int(self.boulder_count_var.get())
        for i in range(selected_boulders):
            self.boulder_widgets[i][1].delete(0, tk.END)
            try:
                self.scoretype_vars[i].set('')  #set each button to '' in every iteration
            except Exception as e:
                print(f"Didn't work to clear. {e}")
        self.name_entry.delete(0, tk.END)
    
    # Manipulating the Climbers and Leaderboards classes
    def add_climber(self):
        """Collects entered information (entry fields). Tries to parse attempts/levels from entry fields and then assigns correct score.
            Updates the leaderboard ranking with the climbers name and their total score."""
        self.enable_edits()
        try:
            climber_name = self.name_entry.get().strip()
            if not climber_name:
                messagebox.showerror("Name Error", "Climber name field cannot be empty.")
                return

            # Checks if the name already exists in the leaderboard
            for climber in self.leaderboard.climbers:
                if climber.name == climber_name:
                    messagebox.showerror("Error", f"{climber_name} already exists in the leaderboard.\nPlease add another identifier.")
                    return

            try:      # check to see if number selected for boulder_count      
                selected_boulders = int(self.boulder_count_var.get())
            except ValueError:
                messagebox.showerror("Error", "You must select a valid number of boulders to score.")
                return
            # Input validation
            self.validate_scores() 
            boulder_list = []
            for i in range(selected_boulders):
                attempts_entry = self.boulder_widgets[i][1]
                attempts_str = attempts_entry.get()
                level_str = self.scoretype_vars[i].get()
                if attempts_str in ["0"]:
                    attempts = 0.0
                    level = "0"
                else:
                    attempts = int(attempts_str)
                    level = level_str if level_str else "0"
                boulder_list.append(self.Boulder(attempts, level))
            climber = self.Climber(climber_name)
            climber.boulder_list = boulder_list
            self.leaderboard.add_climber(climber)
            messagebox.showinfo("Success", f"Scores for {climber_name} added!")
            self.update_leaderboard()
        except Exception as e:
            print(f"{e}")

    def remove_climber(self):
        """Removes a climber from the Leaderboard"""
        try:
            name = self.name_entry.get()
            if not name:
                raise ValueError(f"Climber name cannot be empty")
            found = False
            for climber in self.leaderboard.climbers[:]:
                if climber.name == name:
                    self.leaderboard.climbers.remove(climber)
                    found = True
                    print(f"{name} has been deleted")
                    break
            if found:
                messagebox.showinfo("Success", f"{name} has been removed.")
                self.update_leaderboard()
            else:
                raise ValueError(f"Climber {name} not found in the leaderboard.")
        except Exception as e:
            messagebox.showerror("Error", f"{e}")

    def edit_climber(self):
        selected_boulders = int(self.boulder_count_var.get())
        try:
            name = self.name_entry.get().strip()
            if not name:
                raise ValueError("Climber name cannot be empty")
            try:
                selected_boulders = int(self.boulder_count_var.get())
            except:
                messagebox.showerror("Boulder Error", "Please select a valid number of boulders.")
                return
            self.validate_scores() 

            # Finding the climber
            climber = next((c for c in self.leaderboard.climbers if c.name == name), None)
            if climber is None:
                raise ValueError(f"Climber '{name}' not found in the leaderboard.")
            
            # Update/Append boulders based on selected entry fields
            for i in range(selected_boulders):
                attempts_entry = self.boulder_widgets[i][1]
                attempts_str = attempts_entry.get()
                level_str = self.scoretype_vars[i].get()
                print(f"level_str = {level_str}")

                if not attempts_str or level_str in ["", "0"]:
                    continue

                try:
                    attempts = int(attempts_str)
                    print(f"{attempts}")
                    level_str in ["25", "10", "5"]
                except(ValueError, IndexError):
                    print("Something went wrong in gathering attempts and attempts level")
                    messagebox.showerror("Value Error", f"Invalid input in Boulder {i + 1}: '{attempts_entry}'")
                    return
                
                boulder = self.Boulder(attempts, level_str)
                if i < len(climber.boulder_list):
                    climber.boulder_list[i] = boulder
                else:
                    climber.boulder_list.append(boulder)
            
            if len(climber.boulder_list) > selected_boulders:
                climber.boulder_list = climber.boulder_list[:selected_boulders]

            self.update_leaderboard()
            messagebox.showinfo("Success", f"Scores for '{name}' were updated sucessfully!")
        except Exception as e:
            messagebox.showerror("Error", f"Error updating climber '{name}':\n{str(e)}")

    # Edits to leaderboard
    def update_leaderboard(self):
        """Clears the leaderboard, re-ranks climbers, and re-prints the leaderboard."""
        self.enable_edits()
        self.leaderboard.rank_climbers()
        self.leaderboard_text.delete(1.0, tk.END)
        self.leaderboard_text.insert(tk.END, str(self.leaderboard))
        self.clear_entries()
        self.disable_edits()    

    def validate_scores(self):
        """Validates the scores in the entry fields, making sure formats are acceptable before submission.
            Specifically does NOT edit or save any information.
            Raises errors if formatting is not accepted."""
        try: 
            selected_boulders = int(self.boulder_count_var.get())       # Check if the variable is an integer.
        except:
            messagebox.showerror("Selection Error", "Please select the number of boulders to score.")
            return
        
        for i in range(selected_boulders):
            
            # Check that there is a selection for each radio button frame.
            level_str = self.scoretype_vars[i].get()
            if not level_str:
                messagebox.showerror("Scoring Error", f"Please select a score type on Boulder {i + 1}.")
                return

            attempts =  self.boulder_widgets[i][1].get()
            try:
                check_attempts = int(attempts)
                check_attempts >= 1
            except:
                messagebox.showerror("Attempts Error", "Make sure the attempts field is filled out, and is at least 1.")
                return

    def clear_leaderboard_ask(self):
        """Asks for confirmation before clearing the entries."""
        response = messagebox.askyesno("Clear Leaderboard", "Are you sure you want to clear the leaderboard?")
        if response:
            self.leaderboard.climbers = []
            self.enable_edits()
            self.leaderboard_text.delete(1.0, tk.END)
            self.disable_edits()
            messagebox.showinfo("Success", "Leaderboard has been cleared.")

    def toggle_score_breakdown(self, show_breakdown):
        self.leaderboard.show_score_breakdown = show_breakdown
        self.update_leaderboard()

    def enable_edits(self):
        """Enables editing of the leaderboard."""
        self.leaderboard_text.config(state= tk.NORMAL)

    def disable_edits(self):
        """Disables editing of the leaderboard."""
        self.leaderboard_text.config(state= tk.DISABLED)


class ScoringIFSC25(tk.Frame):
    """New 2025 IFSC scoring that uses two different levels (top worth 25 points and zone worth 10 points) - ((attempts - 1) * 0.1)"""
    class Leaderboard:
        """Leaderboard class that will visualize ranking of climbers based on their scores."""
        def __init__(self):
            self.climbers = []

        def add_climber(self, climber):
            """Adds a climber to the leaderboard."""
            self.climbers.append(climber)
            
        def rank_climbers(self):
            """Sorts climbers by total score, in descending order."""
            self.climbers.sort(key= lambda climber: climber.total_score(), reverse= True)
            tie_count = 0
            previous_score = None
            current_rank = 1
            tolerance = 1e-5

            for climber in self.climbers:       # New version no longer needs to reference i or enumerate over self.climbers since variables are now used.
                current_score = round(climber.total_score(), 5)

                if previous_score is not None and abs(current_score - previous_score) < tolerance:
                    climber.rank = current_rank
                    tie_count += 1
                else:
                    current_rank = current_rank + tie_count
                    climber.rank = current_rank
                    tie_count = 1
                previous_score = current_score

        def __str__(self):
            """Generate a string representation of the leaderboard."""
            return "\n".join([f"{climber.rank}. {climber.name}: {climber.total_score():.1f} total points"
                      for climber in self.climbers])

    def __init__(self, parent, controller):
        super().__init__(parent)
        # Main Frame
        self.main_frame = ttk.Frame(self)
        self.main_frame.grid(column= 0, row= 0, padx= 20, pady= 20, sticky= "nsew")
        self.main_frame.rowconfigure(0, weight= 1)
        self.main_frame.columnconfigure(0, weight= 1)
        self.main_frame.columnconfigure(1, weight= 1)
        self.leaderboard = self.Leaderboard()

        self.max_boulders = 11
        self.boulder_widgets = []

        self.create_left_frame()
        self.create_right_frame()

        backbutton = ttk.Button(self, text= "Back", command= lambda: self.backbutton_popup(controller))
        backbutton.grid(column= 0, row= 1, columnspan= 2, sticky= "s")

    def backbutton_popup(self, controller):
        """Reminds the user that data isn't shared, then brings the StartingWindow frame to the top."""
        if messagebox.askyesno("Warning!", "Are you sure you want to go back?\nData isn't shared between scoring styles."):
            controller.show_frame("StartingWindow")
        else:
            pass        

    def create_left_frame(self):
        """Create left frame where labels and entry fields exist to gather information from the user."""
        self.left_frame = ttk.Frame(self.main_frame)
        self.left_frame.grid(row=0, column=0, padx= 10, pady= 5, sticky= 'nw')

        # Resizing parameters
        for i in range(0, 3):
            self.left_frame.columnconfigure(i, weight= 1)

        for i in range(0, 20):
            self.left_frame.rowconfigure(i, weight= 1)

        # Name Label and Entry
        self.name_label = ttk.Label(self.left_frame, text= "Climber Name : ")
        self.name_label.grid(column= 0, row= 1, padx=5, pady=5, sticky="e")
        self.name_entry = ttk.Entry(self.left_frame, justify= tk.CENTER)
        self.name_entry.grid(column= 1, row= 1, padx=5, pady=5, sticky="w")

        # Dropdown Menu for Number of Boulders

        self.boulder_count_var = tk.StringVar()
        self.boulder_count_select = ttk.Combobox(
            self.left_frame, textvariable= self.boulder_count_var, state= "readonly",
            values= [str(i) for i in range(1, self.max_boulders)], takefocus= 0
        )
        self.boulder_count_select.set("Number of Boulders")
        self.boulder_count_select.bind("<<ComboboxSelected>>", self.update_boulder_fields) 
        self.boulder_count_select.grid(column= 1, row= 0, columnspan= 2, pady= 10)


        # Creating all widgets as if max number of boulders is selected
        for i in range(self.max_boulders):
            label = ttk.Label(self.left_frame, text=f"Boulder {i + 1} :")
            entry = ttk.Entry(self.left_frame, width= 20, justify= tk.CENTER)
            label.grid(column=0, row=i + 2, padx=5, pady=5)
            entry.grid(column=1, row=i + 2, padx=5, pady=5, sticky="w")
            label.grid_remove()  # Hide initially
            entry.grid_remove()  # Hide initially
            self.boulder_widgets.append((label, entry))


        # Grey text for entry instructions
        self.instruction_label = ttk.Label(
            self.left_frame, 
            foreground= "gray",  
            text= "Enter attempts to the highest scored level. \n    3T = 3 attempts to Top\n    10Z = 10 attempts to Zone"
        )
        self.instruction_label.grid(column= 0, columnspan= 2, row= self.max_boulders + 1)

        # Left Frame buttons
        self.add_climber_button = ttk.Button(self.left_frame, text= "Add Climber", command= lambda: self.add_climber())
        self.add_climber_button.grid(column= 0, columnspan= 2, row= self.max_boulders + 2, padx=5, pady=5)

        self.edit_climber_button = ttk.Button(self.left_frame, text= "Edit Climber", command= lambda: self.edit_climber())
        self.edit_climber_button.grid(column= 0, row= self.max_boulders + 3, sticky= 'w', padx=5, pady=5)

        self.remove_climber_button = ttk.Button(self.left_frame, text= "Remove Climber", command= lambda: self.remove_climber())
        self.remove_climber_button.grid(column= 1, row= self.max_boulders + 3, sticky= 'e', padx=5, pady=5)

    def update_boulder_fields(self, event= None):
        """Show/Hide boulder label/entry fields based on selected number of boulders."""
        selected_count = int(self.boulder_count_var.get())
        # Show the required number of boulder fields
        for i, (label, entry) in enumerate(self.boulder_widgets):
            if i < selected_count:
                label.grid()
                entry.grid()
            else:
                label.grid_remove()
                entry.grid_remove()
        self.left_frame.update_idletasks()

    def create_right_frame(self):
        """Create the right frame for the leaderboard display and associated buttons.
        Should be the same for all scoring types."""
        self.right_frame = ttk.Frame(self.main_frame)
        self.right_frame.grid(row= 0, column= 1, padx= 10, sticky= 'nsew')

        # Resizing parameters
        for i in range(0, 3):
            self.right_frame.columnconfigure(i, weight= 1)

        for i in range(0, 12):
            self.right_frame.rowconfigure(i, weight= 1)

        # Leaderboard Display
        self.leaderboard_text = tk.Text(self.right_frame, height= 20, width= 40)
        self.leaderboard_text.grid(row=0, column=0, columnspan= 2, pady= 10, sticky= "nse")
        self.disable_edits()

        # Rightside Buttons
        self.leaderboard_buttons = ttk.Frame(self.right_frame)
        self.leaderboard_buttons.grid(row=1, column=0, columnspan=2, pady=10)
        ttk.Button(self.leaderboard_buttons, text="Clear Leaderboard", command= self.clear_leaderboard_ask).pack(side= "left", padx= 10, pady= 10) # 

    # Button and Functionality
    def clear_entries(self):
        """Clears the entry fields, saves the number of boulders selected."""
        selected_boulders = int(self.boulder_count_var.get())
        for i in range(selected_boulders):
            self.boulder_widgets[i][1].delete(0, tk.END)
        self.name_entry.delete(0, tk.END)
            
    def add_climber(self):
        """Collects entered information (entry fields). Tries to parse attempts/levels from entry fields and then assigns correct score.
            Updates the leaderboard ranking with the climbers name and their total score."""
        self.enable_edits()
        try:
            climber_name = self.name_entry.get().strip()
            if not climber_name:
                raise ValueError("Climber name field cannot be empty.")
            
            # Checks if the name already exists in the leaderboard
            for climber in self.leaderboard.climbers:
                if climber.name == climber_name:
                    messagebox.showerror("Error", f"{climber_name} already exists in the leaderboard.\nPlease add another identifier.")
                    return

            try:      # check to see if number selected for boulder_count      
                selected_boulders = int(self.boulder_count_var.get())
            except ValueError:
                messagebox.showerror("Error", "You must select a valid number of boulders to score.")
                return
            
            self.validate_scores()  # Checking that scores are entered correctly

            boulders = []
            for i in range(selected_boulders):
                attempts_level = self.boulder_widgets[i][1].get().strip()   # looping through, gathering entry info about the "i"th field and remvoing spaces
                if attempts_level in ["", "0"]:
                    attempts = 0.0
                    level = "0"
                else:
                    attempts = int(attempts_level[:-1]) # gather everyting except last input
                    level = attempts_level[-1].upper()  # Gather the letter, in uppercase
                boulders.append(self.Boulder(attempts, level))
                   
            climber = self.Climber(climber_name)
            climber.boulder_list = boulders
            self.leaderboard.add_climber(climber)

            messagebox.showinfo("Success", f"Scores for {climber_name} added!")
            self.update_leaderboard()

        except Exception as e:
            print(f"{e}")

    def enable_edits(self):
        """Enables editing of the leaderboard."""
        self.leaderboard_text.config(state= tk.NORMAL)

    def disable_edits(self):
        """Disables editing of the leaderboard."""
        self.leaderboard_text.config(state= tk.DISABLED)



    def edit_climber(self):
        """Edits the score of an existing climber. Any spaces outside of the name are stripped but the names are not case sensitive.
            If a new value is entered in the field, the leaderboard gets updated."""
        
        selected_boulders = int(self.boulder_count_var.get()) 
        try:
            name = self.name_entry.get().strip()
            if not name:
                raise ValueError("Climber name cannot be empty")
            
            try:
                selected_boulders = int(self.boulder_count_var.get())
            except: 
                messagebox.showerror("Error", "Please select a valid number of boulders")
                return
            
            self.validate_scores()

            # Find the climber
            climber = next((c for c in self.leaderboard.climbers if c.name == name), None)
            if climber is None:
                raise ValueError(f"Climber '{name}' not found in the leaderboard.")
            
            # Update or append boulders based on selected entry fields
            for i in range(selected_boulders):
                entry_text = self.boulder_widgets[i][1].get()
                if entry_text in [""]:
                    continue

                try:
                    attempts = int(entry_text[:-1])
                    level = entry_text[-1].upper()
                except(ValueError, IndexError):
                    messagebox.showerror("Error", f"Invalid input in Boulder {i + 1}: '{entry_text}'")
                    return
            
                boulder = self.Boulder(attempts, level)
                if i < len(climber.boulder_list):
                    climber.boulder_list[i] = boulder
                else:
                    climber.boulder_list.append(boulder)

            # Matches scored boulders to the current number selected boulders
            if len(climber.boulder_list) > selected_boulders:
                climber.boulder_list = climber.boulder_list[:selected_boulders]

            self.update_leaderboard()
            messagebox.showinfo("Success", f"Scores for '{name}' were updated sucessfully!")
        except Exception as e:
            messagebox.showerror("Error", f"Error updating climber '{name}':\n{str(e)}")    

    def validate_scores(self):
        """Validates the scores in the entry fields, making sure formats are acceptable before submission.
            Specifically does NOT edit or save any information.
            Raises errors if formatting is not accepted."""
        selected_boulders = int(self.boulder_count_var.get()) 

        for i in range(selected_boulders):
            entry_text = self.boulder_widgets[i][1].get().strip().upper()
            # Don't do something for in an entry_text is blank

            if not entry_text or entry_text == "0":
                continue # No point achieved entries are allowed

            if not any(char in entry_text for char in ["T", "Z"]):
                messagebox.showerror("Invalid Input", f"Invalid input '{entry_text}'. Score must end with \n'T' for top, 'Z' for zone, or 0 for no score")
                raise ValueError(f"Invalid input, non-valid level in entry: {i + 1}")
            
            try:
                attempts = int(entry_text[:-1])
            except:
                messagebox.showerror("Invalid Entry", f"Attempts in Boulder {i + 1}: ({entry_text[:-1]}) \nattempts must be a whole number.")
                raise ValueError(f"Invalid entry, in boulder {i + 1}")
            
            if attempts < 0:
                messagebox.showerror("Invalid Entry", f"Attempts cannot be negative in Boulder {i + 1}.")
                raise ValueError(f"Invalid input, negative attempts in entry: {i + 1}")
        
    def remove_climber(self):
        """Removes a climber from the Leaderboard"""
        try:
            name = self.name_entry.get()

            if not name:
                raise ValueError(f"Climber name cannot be empty")
            found = False

            for climber in self.leaderboard.climbers[:]:
                if climber.name == name:
                    self.leaderboard.climbers.remove(climber)
                    found = True
                    print(f"{name} has been deleted")
                    break
            if found:
                messagebox.showinfo("Success", f"{name} has been removed.")
                self.update_leaderboard()
  
            else:
                raise ValueError(f"Climber {name} not found in the leaderboard.")
        except Exception as e:
            messagebox.showerror("Error", f"{e}")

    def clear_leaderboard_ask(self):
        """Asks for confirmation before clearing the entries."""
        response = messagebox.askyesno("Clear Leaderboard", "Are you sure you want to clear the leaderboard?")
        if response:
            self.leaderboard.climbers = []
            self.enable_edits()
            self.leaderboard_text.delete(1.0, tk.END)
            self.disable_edits()
            messagebox.showinfo("Success", "Leaderboard has been cleared.")

    def update_leaderboard(self):
        """Clears the leaderboard, re-ranks climbers, and re-prints the leaderboard."""
        print("Update_leaderboard called")
        self.enable_edits()
        self.leaderboard.rank_climbers()
        self.leaderboard_text.delete(1.0, tk.END)
        self.leaderboard_text.insert(tk.END, str(self.leaderboard))
        self.clear_entries()
        self.disable_edits()

# Logic for ScoringIFSC25
    class Boulder:
        """Represents a single boulder and the attempts to a scoring level, as well as the calculated score.
            * attempts: Number of attempts made to highest scored space.
            * level: Highest achieved scoring space (T for top, Z for zone, 0 for no score)"""
    
        def __init__(self, attempts, level):
            self.attempts = attempts
            self.level = level.upper() if level else "0"
            self.score = self.calculate_score()

        def calculate_score(self):
            """Calculates the score for the boulder based on the highest scored level and attempts to that level.
                * return: calculated score."""
            if self.level == "T":
                return round(25 - ((self.attempts - 1) * 0.1), 5) # using attempts -1 because the score technically only subtracts .1 per "failed" attempt
            elif self.level == "Z":
                return round(10 - ((self.attempts - 1) * 0.1), 5)
            elif self.level == "" or self.level == "0":
                return 0

    class Climber:
        def __init__(self, name):
            self.name = name
            self.boulders = []
            self.rank = None    # Adding rankings rather than just listing order

        def add_boulder(self, attempts, level):
            """Adds the score of a boulder to the climber's scorecard.
                * level: Highest scoring level achieved. ("T", "Z", "0")"""
            self.boulders.append(ScoringIFSC25.Boulder(attempts, level))

        def total_score(self):
            """Calculates the total score of the climber. Used for ranking on the leaderboard."""
            total = round(sum(boulder.score for boulder in self.boulders), 5)

            return total
        
        def delete(self, leaderboard):
            """Deletes the climber from the leaderboard."""
            leaderboard.climbers.remove(self)
        
        def __str__(self):
            return f"{self.name}: {self.total_score():.1f} total points."

if __name__ == "__main__":
    app = App()
    app.mainloop()

Updating Boulder Fields. Max boulders: 11
Updating Boulder Fields. Max boulders: 11
add_climber called
Climbers name: xav
boulder count raw: 1
selected boulders: 1
Boulder 1: attempts='2', level='Z'
Creating climber: xav, with 1 boulders
xav rank 1
before update_leaderboard
before update_leaderboard


In [ ]:
# Current main branch code for v1.0
import tkinter as tk
from tkinter import ttk, messagebox

# OOP Scoring System logic
class Scores:
    class Top:
        def __init__(self, tops = 0, attempts = 0):
            self.tops = tops
            self.attempts = attempts

        def __str__(self):
            return f"{self.tops} T    {self.attempts} A"
    
    class Zone:
        def __init__(self, zones = 0, attempts = 0):
            self.zones = zones
            self.attempts = attempts

        def __str__(self):
            return f"{self.zones} Z    {self.attempts} A"

    class LowZone:
        def __init__(self, low_zones = 0, attempts = 0):
            self.low_zones = low_zones
            self.attempts = attempts

        def __str__(self):
            return f"{self.low_zones} LZ    {self.attempts} A"

    def __init__(self):
        self.tops = Scores.Top(0, 0)
        self.zones = Scores.Zone(0, 0)
        self.low_zones = Scores.LowZone(0, 0)

class Climber:
    def __init__(self, name, category):
        self.name = name
        self.scores = Scores()
        self.category = category

    def __str__(self):
        return f"{self.name}:\n{self.scores.tops}\n{self.scores.zones}\n{self.scores.low_zones}"
    
    def delete(self, leaderboard):
        """Deletes the climber from the leaderboard"""
        leaderboard.climbers.remove(self)
    
class Leaderboard():
    def __init__(self):
        self.climbers = []

    def add_climber(self, climbers):
        """Adds a climber to the leaderboard"""
        self.climbers.append(climbers)
    
    def rank_climbers(self):
        """Sorts climbers by their scores. Key currently matches the USA Climbing Isolation scoring system."""
        def sort_key(climber):
            return (-climber.scores.tops.tops, -climber.scores.zones.zones, -climber.scores.low_zones.low_zones, 
            climber.scores.tops.attempts, climber.scores.zones.attempts, climber.scores.low_zones.attempts)
        self.climbers.sort(key=sort_key)
    
def create_climber(name, category, tops = (0, 0), zones = (0, 0), low_zones = (0, 0)):
    """Creates a climber object with the given name, category, and scores. Scores are taken as tuples.
        If no Score is provided, the name is added to the leaderboard with 0,0"""
    climber = Climber(name, category)
    climber.scores.tops = Scores.Top(*tops)
    climber.scores.zones = Scores.Zone(*zones)
    climber.scores.low_zones = Scores.LowZone(*low_zones)
    return climber

# tkinter window
class ScoringApp:
    def __init__(self, root):
        self.root = root
        self.root.title("Climbing Leaderboard")
        self.leaderboard = Leaderboard()
        
        # Main Frame
        self.main_frame = ttk.Frame(self.root)
        self.main_frame.grid(column= 0, row= 0, padx= 20, pady= 20, sticky= "nsew")
        self.main_frame.rowconfigure(0, weight= 1)
        self.main_frame.columnconfigure(0, weight= 1)
        self.main_frame.columnconfigure(1, weight= 1)

        # Resizing to fit the frame
        self.root.columnconfigure(0, weight= 1)
        self.root.rowconfigure(0, weight= 1)

        # Creating Left and Right sections
        self.create_left_frame()
        self.create_right_frame()

    def create_left_frame(self):
        """Create the left frame for inputs and buttons"""
        self.left_frame = ttk.Frame(self.main_frame)
        self.left_frame.grid(row=0, column=0, padx=10, pady= 5, sticky= 'nw')

        # Left Frame Resizing
        #self.main_frame.columnconfigure(0, weight= 1)
        self.left_frame.columnconfigure(0, weight= 1)
        self.left_frame.columnconfigure(1, weight= 1)
        self.left_frame.rowconfigure(0, weight= 1)
        self.left_frame.rowconfigure(5, weight= 1)

        # Labels and Entry Fields
        labels = ["Climber Name", "Category", "Tops (count, attempts)", "Zones (count, attempts)", "Low Zones (count, attempts)"]
        self.entries = {}
        for i, label in enumerate(labels):
            ttk.Label(self.left_frame, text= label + ":").grid(row=i, column=0, padx=5, pady=5)
            entry = ttk.Entry(self.left_frame)
            entry.grid(row= i, column= 1, padx=5, pady=5, sticky= "w")
            self.entries[label] = entry #stores in dictionary for easy access
        
        # Leftside Buttons
        ttk.Button(self.left_frame, text="Add Climber", command=self.add_climber).grid(row= 5, column= 0, columnspan= 2, pady= 10)

        # Edit/Remove Climber Buttons
        add_remove_buttons = [
            ("Edit Climber", self.edit_climber),
            ("Remove Climber", self.remove_climber)
        ]
        for i, (text, command) in enumerate(add_remove_buttons):
            ttk.Button(self.left_frame, text= text, command= command).grid(row= 6, column= i, columnspan= 2, padx= 10, pady= 10, sticky= "w")

    def create_right_frame(self):
        """Create the right frame for the leaderboard display and associated buttons"""
        self.right_frame = ttk.Frame(self.main_frame)
        self.right_frame.grid(row=0, column=1, padx=10, sticky= 'nsew')

        #letting right frame expand to fill the space
        self.main_frame.columnconfigure(1, weight= 1)
        self.right_frame.rowconfigure(0, weight= 1)
        self.right_frame.columnconfigure(0, weight= 1)

        # Leaderboard Display
        self.leaderboard_text = tk.Text(self.right_frame, height=20, width=30)
        self.leaderboard_text.grid(row=0, column=0, columnspan= 2, pady=10, sticky= "nsew")
        self.disable_edits()

        # Rightside Buttons
        self.leaderboard_buttons = ttk.Frame(self.right_frame)
        self.leaderboard_buttons.grid(row=1, column=0, columnspan=2, pady=10)
        ttk.Button(self.leaderboard_buttons, text="Clear Leaderboard", command=self.clear_leaderboard_ask).pack(side= "left", padx= 10, pady= 10)
        ttk.Button(self.leaderboard_buttons, text="Show Leaderboard", command=self.show_leaderboard).pack(side= "left", padx= 10, pady= 10)

    # Functions and Logic
    def clear_entries(self):
        """Clears all entry fields"""
        for entry in self.entries.values():
            entry.delete(0, tk.END)

    def remove_climber(self):
        """Removes a climber from the Leaderboard"""
        try:
            name = self.entries["Climber Name"].get()
            found = False
            if name == "":
                raise ValueError("Climber Name cannot be empty")
            for climber in self.leaderboard.climbers[:]:
                if climber.name == name:
                    climber.delete(self.leaderboard)
                    found = True
                    break
            if found:
                messagebox.showinfo("Success", f"{name} has been removed.")
                self.clear_entries()
                self.show_leaderboard()
            else:
                raise ValueError(f"Climber {name} not found in the leaderboard.")
        except Exception as e:
            messagebox.showerror("Error", str(e))

    def validate_scores(self, score_type):
        """Parses and validates the score from the entry field. Raises error if an incorrect score is give"""
        entry_value = self.entries[score_type].get()
        scores = tuple(map(int, entry_value.split(',')))
        if not entry_value:
            return (0, 0)
        short_types = score_type.split()[0]
        if scores[0] > scores[1]:
            raise ValueError(f"{short_types}: '{scores[0]}' cannot be larger than attempts '{scores[1]}'")
        return scores

    def add_climber(self):
        """Adds a climber to the current leaderboard. Raises errors if the input is invalid"""
        try:
            name = self.entries["Climber Name"].get()
            if name == "":
                raise ValueError("Name cannot be empty")
            if any(climber.name == name for climber in self.leaderboard.climbers):
                raise ValueError(f"{name} already exists in the leaderboard")
            category = self.entries["Category"].get()
            tops = self.validate_scores("Tops (count, attempts)")
            zones = self.validate_scores("Zones (count, attempts)")
            low_zones = self.validate_scores("Low Zones (count, attempts)") if self.entries["Low Zones (count, attempts)"].get() else (0, 0)
   
            climber = create_climber(name, category, tops, zones, low_zones)
            self.leaderboard.add_climber(climber)
            messagebox.showinfo("Success", f"{name} added to the leaderboard!")
            self.clear_entries()
            self.show_leaderboard()
        except Exception as e:
            messagebox.showerror("Error", f"Invalid input: {e}")

    def show_leaderboard(self):
        """Displays the current leaderboard, with rankings"""
        self.leaderboard.rank_climbers()
        self.enable_edits()
        self.leaderboard_text.delete(1.0, tk.END)
        for climber in self.leaderboard.climbers:
            self.leaderboard_text.insert(tk.END, f"{climber}\n\n")
        self.disable_edits()

    def edit_climber(self):
        """Edits the score of an existing climber.
            If a new value is entered in the entry field, it is validated and updated"""
        try: 
            name = self.entries["Climber Name"].get()  
            if not name:
                raise ValueError("Climber Name cannot be empty")  
             
            climber = next((c for c in self.leaderboard.climbers if c.name == name), None)
            if not climber:
                raise ValueError(f"Climber {name} not found.")
            
            category = self.entries["Category"].get()
            if category:
                climber.category = category
            
            if self.entries["Tops (count, attempts)"].get():
                tops = self.validate_scores("Tops (count, attempts)")
                climber.scores.tops = Scores.Top(*tops)

            if self.entries["Zones (count, attempts)"].get():
                zones = self.validate_scores("Zones (count, attempts)")
                climber.scores.zones = Scores.Zone(*zones)
        
            if self.entries["Low Zones (count, attempts)"].get():
                low_zones = self.validate_scores("Low Zones (count, attempts)")
                climber.scores.low_zones = Scores.LowZone(*low_zones)

            messagebox.showinfo("Success", f"Climber '{name}' has been updated.")
            self.clear_entries()
            self.show_leaderboard()
        except Exception as e:
            messagebox.showerror("Error", f"Error updating climber '{name}':\n{str(e)}")

    def clear_leaderboard_ask(self):
        """Asks for confirmation before clearing the entries"""
        response = messagebox.askyesno("Clear Leaderboard", "Are you sure you want to clear the leaderboard?")
        if response:
            self.clear_leaderboard()
        
    def clear_leaderboard(self):
        """Clears the leaderboard values"""
        self.leaderboard.climbers = []
        self.enable_edits()
        self.leaderboard_text.delete(1.0, tk.END)
        messagebox.showinfo("Success", "Leaderboard has been cleared.")
        self.disable_edits()

    def enable_edits(self):
        self.leaderboard_text.config(state= tk.NORMAL)

    def disable_edits(self):
        self.leaderboard_text.config(state= tk.DISABLED)

if __name__ == "__main__":
    root = tk.Tk()
    app = ScoringApp(root)
    root.mainloop()

In [ ]:
# Starting from scratch with ScoringOlympic, should be very similar to ScoringIFSC
def scoring_olympic(top = 0, highzone = 0, lowzone = 0, attempts = 0):
    points = 0
    if top > 0:
        points = (25 - ((attempts - 1) * 0.1))
    elif highzone > 0:
        points = (10 - ((attempts - 1) * 0.1))
    elif lowzone > 0:
        points = (5 - ((attempts - 1) * 0.1))
    else:
        points = 0
    return points

print(scoring_olympic(top= 1, attempts = 12))
"""
    There are 3 scored zones in the olympic scoring system:
    *   25 points awarded for achieving the top, 
    *   10 points for the 'high zone', and
    *   5 points for the 'low zone'.
    If no zone is reached, a score of 0 will be given.

    Same as ScoringIFSC, 0.1 is taken off from the boulder score for each FAILED attempt[Scored level - ((attempts - 1) * 0.1)]
"""

class Boulder:
    """Represents a single boulder and the attempts to a scoring level, as well as the calculated score.
        * attempts: Number of attempts made to highest scored space.
        * level: Highest achieved scoring space (T for top, Z for zone, 0 for no score)"""

    def __init__(self, attempts, level):
        self.attempts = attempts
        self.level = level.upper() if level else "0"
        self.score = self.calculate_score()

    def calculate_score(self):
        """Calculates the score for the boulder based on the highest scored level and attempts to that level.
            * return: calculated score."""
        if self.level == "T":
            return round(25 - ((self.attempts - 1) * 0.1), 5) # using attempts -1 because the score technically only subtracts .1 per "failed" attempt
        elif self.level == "Z":
            return round(10 - ((self.attempts - 1) * 0.1), 5)
        elif self.level == "LZ":
            return round(5 - ((self.attempts - 1) * 0.1), 5)
        elif self.level == "" or self.level == "0":
            return 0

class Climber:
    def __init__(self, name):
        self.name = name
        self.boulders = []
        self.rank = None    # Adding rankings rather than just listing order

    def add_boulder(self, attempts, level):
        """Adds the score of a boulder to the climber's scorecard.
            * level: Highest scoring level achieved. ("T", "Z", "0")"""
        self.boulders.append(ScoringIFSC25.Boulder(attempts, level))

    def total_score(self):
        """Calculates the total score of the climber. Used for ranking on the leaderboard."""
        total = round(sum(boulder.score for boulder in self.boulders), 5)

        return total
    
    def delete(self, leaderboard):
        """Deletes the climber from the leaderboard."""
        leaderboard.climbers.remove(self)
    
    def __str__(self):
        return f"{self.name}: {self.total_score():.1f} total points."
    
class Leaderboard:
        """Leaderboard class that will visualize ranking of climbers based on their scores."""
        def __init__(self):
            self.climbers = []

        def add_climber(self, climber):
            """Adds a climber to the leaderboard."""
            self.climbers.append(climber)
            
        def rank_climbers(self):
            """Sorts climbers by total score, in descending order."""
            self.climbers.sort(key= lambda climber: climber.total_score(), reverse= True)
            tie_count = 0
            previous_score = None
            current_rank = 1
            tolerance = 1e-5

            for climber in self.climbers:       # New version no longer needs to reference i or enumerate over self.climbers since variables are now used.
                current_score = round(climber.total_score(), 5)

                if previous_score is not None and abs(current_score - previous_score) < tolerance:
                    climber.rank = current_rank
                    tie_count += 1
                else:
                    current_rank = current_rank + tie_count
                    climber.rank = current_rank
                    tie_count = 1
                previous_score = current_score

        def __str__(self):
            """Generate a string representation of the leaderboard."""
            return "\n".join([f"{climber.rank}. {climber.name}: {climber.total_score():.1f} total points"
                      for climber in self.climbers])
        
"""
    Go back to this code if using radio buttons.
    def create_left_frame(self):
        # Create left frame where labels and entry fields exist to gather information from the user.
        self.left_frame = ttk.Frame(self.main_frame)
        self.left_frame.grid(row=0, column=0, padx= 10, pady= 5, sticky= 'nw')

        # Resizing parameters
        for i in range(0, 3):
            self.left_frame.columnconfigure(i, weight= 1)

        for i in range(0, 20):
            self.left_frame.rowconfigure(i, weight= 1)

        # Name Label and Entry
        self.name_label = ttk.Label(self.left_frame, text= "Climber Name : ")
        self.name_label.grid(column= 0, row= 1, padx=5, pady=5, sticky="e")
        self.name_entry = ttk.Entry(self.left_frame, justify= tk.CENTER)
        self.name_entry.grid(column= 1, row= 1, padx=5, pady=5, sticky="w")

        # Dropdown Menu for Number of Boulders
        self.boulder_count_var = tk.StringVar()
        self.boulder_count_select = ttk.Combobox(
            self.left_frame, textvariable= self.boulder_count_var, state= "readonly",
            values= [str(i) for i in range(1, self.max_boulders)], takefocus= 0
        )
        self.boulder_count_select.set("Number of Boulders")
        self.boulder_count_select.bind("<<ComboboxSelected>>", self.update_boulder_fields) 
        self.boulder_count_select.grid(column= 1, row= 0, columnspan= 2, pady= 10)

        score_combobox_options = ['25', '10', '5', '0']
        # Creating all widgets as if max number of boulders is selected
        for i in range(self.max_boulders):
            label = ttk.Label(self.left_frame, text=f"Boulder {i + 1} Attempts:")
            entry = ttk.Entry(self.left_frame, width= 20, justify= tk.CENTER)

            score_var = tk.StringVar(value= "")
            radio_frame = tk.Frame(self.left_frame)
            #combobox = tk.Radiobutton(self.left_frame, values= score_combobox_options, textvariable= score_var)

            for x, val in enumerate(score_combobox_options):
                rb = tk.Radiobutton(
                    radio_frame, 
                    text= val,
                    variable= score_var,
                    value= val
                )
                rb.grid(row= 0, column= x, padx= 2, pady= 2, sticky= "w")

            label.grid(column=0, row=i + 2, padx=5, pady=5)
            entry.grid(column=1, row=i + 2, padx=5, pady=5, sticky="w")
            radio_frame.grid(column= 3, row= i + 2, padx=5, pady=5, sticky="w")
            self.left_frame.update_idletasks()
            label.grid_remove()  # Hide initially
            entry.grid_remove()  # Hide initially
            radio_frame.grid_remove() # Hide initially
            self.boulder_widgets.append((label, entry, radio_frame))
            self.scoretype_vars.append(score_var)

        # Grey text for entry instructions
        self.instruction_label = ttk.Label(
            self.left_frame, 
            foreground= "gray",  
            text= "Enter attempts to the highest scored level. \nIn the second box, select the highest scored level"
        )
        self.instruction_label.grid(column= 0, columnspan= 2, row= self.max_boulders + 1)
        
        # Left Frame buttons
        self.add_climber_button = ttk.Button(self.left_frame, text= "Add Climber", command= lambda: print("Add climber button"))
        self.add_climber_button.grid(column= 0, columnspan= 2, row= self.max_boulders + 2, padx=5, pady=5)

        self.edit_climber_button = ttk.Button(self.left_frame, text= "Edit Climber", command= lambda: print("Edit climber button"))
        self.edit_climber_button.grid(column= 0, row= self.max_boulders + 3, sticky= 'w', padx=5, pady=5)

        self.remove_climber_button = ttk.Button(self.left_frame, text= "Remove Climber", command= lambda: print("Remove climber button"))
        self.remove_climber_button.grid(column= 1, row= self.max_boulders + 3, sticky= 'e', padx=5, pady=5)
"""



23.9


"\n    There are 3 scored zones in the olympic scoring system:\n    *   25 points awarded for achieving the top, \n    *   10 points for the 'high zone', and\n    *   5 points for the 'low zone'.\n    If no zone is reached, a score of 0 will be given.\n\n    Same as ScoringIFSC, 0.1 is taken off from the boulder score for each FAILED attempt[Scored level - ((attempts - 1) * 0.1)]\n"

In [ ]:
# Starting from scratch with ScoringIFSC25
def perboulder_scoring(
        b1top, b1zone, b1attempts, b2top, b2zone, b2attempts, b3top, b3zone, b3attempts, b4top, b4zone, b4attempts
        ):
    
    if b1top > 0:
        b1score = 25 - (b1attempts * 0.1)
    elif b1top == 0 and b1zone > 0:
        b1score = 10 - (b1attempts * 0.1)
    else:
        b1score = 0

    if b2top > 0:
        b2score = 25 - (b2attempts * 0.1)
    elif b2top == 0 and b2zone > 0:
        b2score = 10 - (b2attempts * 0.1)
    else:
        b2score = 0

    if b3top > 0:
        b3score = 25 - (b3attempts * 0.1)
    elif b3top == 0 and b3zone > 0:
        b3score = 10 - (b3attempts * 0.1)
    else:
        b3score = 0

    if b4top > 0:
        b4score = 25 - (b4attempts * 0.1)
    elif b4top == 0 and b4zone > 0:
        b4score = 10 - (b4attempts * 0.1)
    else:
        b4score = 0
    
    total = b1score + b2score + b3score + b4score

    print(f"B1: {b1score}\nB2: {b2score}\nB3: {b3score}\nB4: {b4score}\nTotal: {total}")

def score_perboulder(*boulders):
    """
    Scores and prints each boulder entered into the function.
    The highest achieved point shall be scored (if both top and zone achieved, score the top)
    Written as ___Attempts to scored area___Scored Area:
        xT = Top in x attempts
        xZ = Zone in x attempts
        0 = No score achieved 
    """
    total_score = 0
    scores = []

    # For each boulder given in the function, do the following:
    for i, boulder in enumerate(boulders, start= 1):
        if boulder == "0":
            score = 0
        elif not any(char in boulder.upper() for char in ["T", "Z"]):
            score = 0
        else:
            attempts = int(boulder[:-1])        # Extraction of attempts
            result = boulder[-1].upper()        # Extraction of score type

            if result == "T":
                score = 25 - (attempts * 0.1)
            elif result == "Z":
                score = 10 - (attempts * 0.1)
            else:
                score = 0                       # Score for no top/zone
            
        scores.append(score)
        total_score += score
        print(f"B{i}: {score:.1f}")

    print(f"Total: {total_score:.1f}")

class Boulder:
    """Represents a single boulder and the attempts to a scoring level, as well as the calculated score.
        * attempts: Number of attempts made to highest scored space.
        * level: Highest achieved scoring space (T for top, Z for zone, 0 for no score)"""
    
    def __init__(self, attempts, level):
        self.attempts = attempts
        self.level = level.upper() if level else "0"
        self.score = self.calculate_score()

    def calculate_score(self):
        """Calculates the score for the boulder based on the highest scored level and attempts to that level.
            * return: calculated score."""
        if self.result == "T":
            return 25 - (self.attempts * 0.1)
        elif self.result == "Z":
            return 10 - (self.attempts * 0.1)
        else:
            return 0        

    def __str__(self):
        pass

class Climber:
    def __init__(self, name):
        self.name = name
        self.boulders = []

    def add_boulder(self, attempts, level):
        """Adds the score of a boulder to the climber's scorecard.
            * name: Name of the climber
            * level: Highest scoring level achieved. ("T", "Z", "0")"""
        self.boulders.append(Boulder(attempts, level))

    def total_score(self):
        """Calculates the total score of the climber. Used for ranking on the leaderboard."""
        return sum(boulder.score for boulder in self.boulders)
    
    def __str__(self):
        pass

class Leaderboard:
    """Leaderboard class that will visualize ranking of climbers based on their scores."""
    def __init__(self):
        self.climbers = []

    def add_climber(self, climber):
        """Adds a climber to the leaderboard."""
        self.climbers.append(climber)

    def rank_climbers(self):
        """Sorts climbers by total score, in descending order."""
        self.climbers.sort(key= lambda climber: climber.total_score(), reverse= True)

    def __str__(self):
        return "\n".join([f"{i + 1}. {climber.name}: {climber.total_score():.1f} total points" for i, climber in enumerate(self.climbers)])

#perboulder_scoring(0, 1, 4, 1, 0, 12, 0, 1, 4, 0, 0, 0)

#score_perboulder("4Z", "12t", "4z", "0")


B1: 9.6
B2: 23.8
B3: 9.6
B4: 0
Total: 43.0
B1: 9.6
B2: 23.8
B3: 9.6
B4: 0.0
Total: 43.0


In [ ]:
# Using Polymorphism with the scoring types.
class ScoringStyle:
    def __init__(self):
        """Initialize the scoring style."""
        pass

    def rank_climbers(self, climber):
        """Sorts climbers based on the scoring logic, which is implemented by the subclass."""
        raise NotImplementedError("Subclass must implement this method.")
    
class USAC_ScoringStyle(ScoringStyle):
    class Scores:
        class Top:
            def __init__(self, tops = 0, attempts = 0):
                self.tops = tops
                self.attempts = attempts

            def __str__(self):
                return f"{self.tops} T    {self.attempts} A"
        
        class Zone:
            def __init__(self, zones = 0, attempts = 0):
                self.zones = zones
                self.attempts = attempts

            def __str__(self):
                return f"{self.zones} Z    {self.attempts} A"

        class LowZone:
            def __init__(self, low_zones = 0, attempts = 0):
                self.low_zones = low_zones
                self.attempts = attempts

            def __str__(self):
                return f"{self.low_zones} LZ    {self.attempts} A"

        def __init__(self):
            self.tops = USAC_ScoringStyle.Scores.Top(0, 0)
            self.zones = USAC_ScoringStyle.Scores.Zone(0, 0)
            self.low_zones = USAC_ScoringStyle.Scores.LowZone(0, 0)

    def rank_climbers(self, climbers):
        """Sorts climbers by their scores. Key currently matches the USA Climbing Isolation scoring system."""
        def sort_key(climber):
            return (-climber.scores.tops.tops, -climber.scores.zones.zones, -climber.scores.low_zones.low_zones, 
            climber.scores.tops.attempts, climber.scores.zones.attempts, climber.scores.low_zones.attempts)
        self.climbers.sort(key=sort_key)

class OlympicScores(ScoringStyle):
    class Top:
        def __init__(self, tops = 0, attempts = 0):
            self.tops = tops
            self.attempts = attempts

        def __str__(self):
            return f"{self.tops} T    {self.attempts} A"
        
        def calculate_score(self):
            return (25 * self.tops) - (max(0, self.attempts - self.tops) * 0.1)
    
    class Zone10:
        def __init__(self, zone10 = 0, attempts = 0):
            self.zone10 = zone10
            self.attempts = attempts

        def __str__(self):
            return f"{self.zone10} Z10    {self.attempts} A"

        def calculate_score(self):
            return (10 * self.zone10) - (max(0, self.attempts - self.zone10) * 0.1)        

    class Zone5:
        def __init__(self, zone5 = 0, attempts = 0):
            self.zone5 = zone5
            self.attempts = attempts

        def __str__(self):
            return f"{self.zone5} Z5    {self.attempts} A"

        def calculate_score(self):
            return (5 * self.zone5) - (max(0, self.attempts - self.zone5) * 0.1)

    class Total:
        def __init__(self, top = None, zone10 = None, zone5 = None):
            self.top = top if top else OlympicScores.Top()
            self.zone10 = zone10 if zone10 else OlympicScores.Zone10()
            self.zone5 = zone5 if zone5 else OlympicScores.Zone5()

            self.total_score = (
                self.top.calculate_score() +
                self.zone10.calculate_score() +
                self.zone5.calculate_score()
            )

        def __str__(self):
            return f"Total Score: {self.total_score: .1f}\n Tops Score: {self.top.calculate_score()}\n Zone10 Score: {self.zone10.calculate_score()}\n Zone5 Score: {self.zone5.calculate_score()}"
        



In [ ]:
# Alternative Scoring System
    # Scoring using the 25/15/10/5 system found in older USAC competitions and some IFSC.
scoring_mode = "Olympic"
'''
Here's how the scores work:
If a climber tops a boulder, they are awarded 25 - ((attempts - 1) * 0.1) points. If top is not achieved, their added attempts/score is awarded.
when a climber reaches each scored space (15/10/5) they are awarded those points, minus .1 for each attempt taken. Only the highest scored value is taken.
With 4 boulders, there is a maximum of 100 points total. If there aren't enough zones for each number, then the higher number is taken.
Ranking is based on total score
'''
# Breakdown of how the scoring should look, Using max here to make sure tops isn't larger than attempts
"""
top_score = (25 * num_of_tops) - (max(0, attempts_to_top - num_of_tops) * 0.1)
zone10_score = (10 * num_of_zone10) - (max(0, attempts_to_zone10 - num_of_zone10) * 0.1)
zone5_score = (5 * num_of_zone5) - (max(0, attempts_to_zone5 - num_of_zone5) * 0.1)
Total_score = top_score + zone10_score + zone5_score
"""


class OlympicScores:
    class Top:
        def __init__(self, tops = 0, attempts = 0):
            self.tops = tops
            self.attempts = attempts

        def __str__(self):
            return f"{self.tops} T    {self.attempts} A"
        
        def calculate_score(self):
            return (25 * self.tops) - (max(0, self.attempts - self.tops) * 0.1)
    
    class Zone10:
        def __init__(self, zone10 = 0, attempts = 0):
            self.zone10 = zone10
            self.attempts = attempts

        def __str__(self):
            return f"{self.zone10} Z10    {self.attempts} A"

        def calculate_score(self):
            return (10 * self.zone10) - (max(0, self.attempts - self.zone10) * 0.1)        

    class Zone5:
        def __init__(self, zone5 = 0, attempts = 0):
            self.zone5 = zone5
            self.attempts = attempts

        def __str__(self):
            return f"{self.zone5} Z5    {self.attempts} A"

        def calculate_score(self):
            return (5 * self.zone5) - (max(0, self.attempts - self.zone5) * 0.1)

    class Total:
        def __init__(self, top = None, zone10 = None, zone5 = None):
            self.top = top if top else OlympicScores.Top()
            self.zone10 = zone10 if zone10 else OlympicScores.Zone10()
            self.zone5 = zone5 if zone5 else OlympicScores.Zone5()

            self.total_score = (
                self.top.calculate_score() +
                self.zone10.calculate_score() +
                self.zone5.calculate_score()
            )

        def __str__(self):
            return f"Total Score: {self.total_score: .1f}\n Tops Score: {self.top.calculate_score()}\n Zone10 Score: {self.zone10.calculate_score()}\n Zone5 Score: {self.zone5.calculate_score()}"

# Written as (scored zones/tops, attempts to zones/tops)
top = OlympicScores.Top(2, 6)
zone10 = OlympicScores.Zone10(3, 8)
zone5 = OlympicScores.Zone5(4, 9)

total = OlympicScores.Total(top, zone10, zone5)
print(total)
  

class Climber:
    def __init__(self, name):
        self.name = name
        self.scores = OlympicScores()

    def __str__(self):
        return f"{self.name}: {self.scores.Total} points."




Total Score:  98.6
 Tops Score: 49.6
 Zone10 Score: 29.5
 Zone5 Score: 19.5


In [3]:
""" NEEDS WORK """
#Teasing out scoring errors. A score should only be earned by the highest point a climber got to in each boulder
#  and then -.1 point for each unsucessful attempt

class OlympicScores:
    class Top:
        def __init__(self, tops = 0, attempts = 0):
            self.tops = tops
            self.attempts = attempts

        def __str__(self):
            return f"{self.tops} T    {self.attempts} A"
        
        def calculate_score(self):
            return (25 * self.tops) - (max(0, self.attempts - self.tops) * 0.1)
    
    class Zone10:
        def __init__(self, zone10 = 0, attempts = 0):
            self.zone10 = zone10
            self.attempts = attempts

        def __str__(self):
            return f"{self.zone10} Z10    {self.attempts} A"

        def calculate_score(self):
            return (10 * self.zone10) - (max(0, self.attempts - self.zone10) * 0.1)        

    class Zone5:
        def __init__(self, zone5 = 0, attempts = 0):
            self.zone5 = zone5
            self.attempts = attempts

        def __str__(self):
            return f"{self.zone5} Z5    {self.attempts} A"

        def calculate_score(self):
            return (5 * self.zone5) - (max(0, self.attempts - self.zone5) * 0.1)

    class Total:
        def __init__(self, top = None, zone10 = None, zone5 = None):
            self.top = top if top else OlympicScores.Top()
            self.zone10 = zone10 if zone10 else OlympicScores.Zone10()
            self.zone5 = zone5 if zone5 else OlympicScores.Zone5()

            self.total_score = (
                self.top.calculate_score() +
                self.zone10.calculate_score() +
                self.zone5.calculate_score()
            )

        def __str__(self):
            return f"Total Score: {self.total_score: .1f}\n Tops Score: {self.top.calculate_score()}\n Zone10 Score: {self.zone10.calculate_score()}\n Zone5 Score: {self.zone5.calculate_score()}"

# Written as (scored zones/tops, attempts to zones/tops)
top = OlympicScores.Top(2, 6)
zone10 = OlympicScores.Zone10(3, 8)
zone5 = OlympicScores.Zone5(4, 9)

total = OlympicScores.Total(top, zone10, zone5)
print(total)
  

class Climber:
    def __init__(self, name):
        self.name = name
        self.scores = OlympicScores()

    def __str__(self):
        return f"{self.name}: {self.scores.Total} points."


#print(f"Total Score: {total_score}")

Total Score:  98.6
 Tops Score: 49.6
 Zone10 Score: 29.5
 Zone5 Score: 19.5


In [ ]:
# Working with a dropdown menu to select which type of scoring system to use
import tkinter as tk
from tkinter import ttk, messagebox

options = ["Olympic Scoring", "Zone Based Scoring", "5/10/15/25 Scoring"]
class TestWindow:
    def __init__(self, root):
        self.root = root
        self.root.title("Testing Default Window")
        self.main_page = ttk.Frame(self.root)
        self.main_page.grid(column= 0, row= 0, padx= 20, pady= 20, sticky= "nsew")
        self.root.geometry("400x400")

        self.selected_value = tk.StringVar(root)
        self.selected_value.set("Choose a scoring type")
        self.dropdown_menu = ttk.OptionMenu(self.main_page, self.selected_value, self.selected_value.get(), *options)
        self.dropdown_menu.grid(column= 0, row= 1, padx= 20, pady= 20)

        self.submit_button = ttk.Button(self.main_page, text= "Submit", command= self.update_frame)
        self.submit_button.grid(column= 0, row= 2)

        self.dynamic_frame = ttk.Frame(self.main_page)
        self.dynamic_frame.grid(column= 0, row= 3, pady= 10)

    def update_frame(self):
        """Updates the dynamic frame based on the selected scoring option."""
        for widget in self.dynamic_frame.winfo_children():
            widget.destroy()

        # Get the selected option
        selected_option = self.selected_value.get()

        if selected_option == "Olympic Scoring":
            OlympicScoring(self.dynamic_frame)
        elif selected_option == "Zone Based Scoring":
            ZoneBasedScoring(self.dynamic_frame)
        else:
            ttk.Label(self.dynamic_frame, text= "Please select a valid option").grid(column=0, row=0, pady=10)

class OlympicScoring:
    """GUI Window for entering information about Olympic Style Scoring"""
    def __init__(self, parent):
        self.parent = parent

        # GUI Input fields for Olympic Scoring
        ttk.Label(parent, text= "Name: ").grid(column= 0, row= 0)
        self.name = ttk.Entry(parent, width= 10).grid(column= 1, row= 0, columnspan= 2)

        ttk.Label(parent, text="Tops (count, attempts):").grid(column=0, row=1, sticky="w")
        self.tops_count = ttk.Entry(parent, width=10)
        self.tops_count.grid(column=1, row=1, padx=5)
        self.tops_attempts = ttk.Entry(parent, width=10)
        self.tops_attempts.grid(column=2, row=1, padx=5)

        ttk.Label(parent, text="Zone10 (count, attempts):").grid(column=0, row=2, sticky="w")
        self.zone10_count = ttk.Entry(parent, width=10)
        self.zone10_count.grid(column=1, row=2, padx=5)
        self.zone10_attempts = ttk.Entry(parent, width=10)
        self.zone10_attempts.grid(column=2, row=2, padx=5)

        ttk.Label(parent, text="Zone5 (count, attempts):").grid(column=0, row=3, sticky="w")
        self.zone5_count = ttk.Entry(parent, width=10)
        self.zone5_count.grid(column=1, row=3, padx=5)
        self.zone5_attempts = ttk.Entry(parent, width=10)
        self.zone5_attempts.grid(column=2, row=3, padx=5)

        # Submit button to calculate the score
        self.calculate_button = ttk.Button(parent, text="Calculate Score", command=self.calculate_score)
        self.calculate_button.grid(column=0, row=4, pady=10, columnspan=3)

        # Label to display the result
        self.result_label = ttk.Label(parent, text="")
        self.result_label.grid(column=0, row=5, pady=10, columnspan=3)

    def calculate_score(self):
        tops = int(self.tops_count.get())
        tops_attempts = int(self.tops_attempts.get())
        zone10 = int(self.zone10_count.get())
        zone10_attempts = int(self.zone10_attempts.get())
        zone5 = int(self.zone5_count.get())
        zone5_attempts = int(self.zone5_attempts.get())

        # storing data for final scores
        top_score = OlympicScores.Top(tops, tops_attempts)
        zone10_score = OlympicScores.Zone10(zone10, zone10_attempts)
        zone5_score = OlympicScores.Zone5(zone5, zone5_attempts)
        total_score = OlympicScores.Total(top_score, zone10_score, zone5_score)

        # Display results
        self.result_label.config(
            text= f"Total Score: {total_score.total_score:.1f}\nScore From Tops: {top_score}\nScore from Zone10: {zone10_score}"
                                  )

class OlympicScores:
    """Logic for Olympic Scores"""
    class Top:
        def __init__(self, tops = 0, attempts = 0):
            self.tops = tops
            self.attempts = attempts

        def __str__(self):
            return f"{self.tops} T    {self.attempts} Attempts"
        
        def calculate_score(self):
            return (25 * self.tops) - (max(0, self.attempts - self.tops) * 0.1)
    
    class Zone10:
        def __init__(self, zone10 = 0, attempts = 0):
            self.zone10 = zone10
            self.attempts = attempts

        def __str__(self):
            return f"{self.zone10} Z10    {self.attempts} Attempts"

        def calculate_score(self):
            return (10 * self.zone10) - (max(0, self.attempts - self.zone10) * 0.1)        

    class Zone5:
        def __init__(self, zone5 = 0, attempts = 0):
            self.zone5 = zone5
            self.attempts = attempts

        def __str__(self):
            return f"{self.zone5} Z5    {self.attempts} Attempts"

        def calculate_score(self):
            return (5 * self.zone5) - (max(0, self.attempts - self.zone5) * 0.1)

    class Total:
        def __init__(self, top = None, zone10 = None, zone5 = None):
            self.top = top if top else OlympicScores.Top()
            self.zone10 = zone10 if zone10 else OlympicScores.Zone10()
            self.zone5 = zone5 if zone5 else OlympicScores.Zone5()

            self.total_score = (
                self.top.calculate_score() +
                self.zone10.calculate_score() +
                self.zone5.calculate_score()
            )

        def __str__(self):
            return f"Total Score: {self.total_score: .1f}\n Tops Score: {self.top.calculate_score()}\n Zone10 Score: {self.zone10.calculate_score()}\n Zone5 Score: {self.zone5.calculate_score()}"


class ZoneBasedScoring:
    def __init__(self, parent):
        ttk.Label(parent, text= "Zone Based Scoring Selected").grid(column= 0, row= 0, pady= 10)

if __name__ == "__main__":
    root = tk.Tk()
    app = TestWindow(root)
    root.mainloop()

In [11]:
# Testing Cell
import tkinter as tk
from tkinter import ttk, messagebox

# OOP Scoring System logic
class Scores:
    class Top:
        def __init__(self, tops = 0, attempts = 0):
            self.tops = tops
            self.attempts = attempts

        def __str__(self):
            return f"{self.tops} T    {self.attempts} A"
    
    class Zone:
        def __init__(self, zones = 0, attempts = 0):
            self.zones = zones
            self.attempts = attempts

        def __str__(self):
            return f"{self.zones} Z    {self.attempts} A"

    class LowZone:
        def __init__(self, low_zones = 0, attempts = 0):
            self.low_zones = low_zones
            self.attempts = attempts

        def __str__(self):
            return f"{self.low_zones} LZ    {self.attempts} A"

    def __init__(self):
        self.tops = Scores.Top(0, 0)
        self.zones = Scores.Zone(0, 0)
        self.low_zones = Scores.LowZone(0, 0)

class Climber:
    def __init__(self, name, category):
        self.name = name
        self.scores = Scores()
        self.category = category

    def __str__(self):
        return f"{self.name}:\n{self.scores.tops}\n{self.scores.zones}\n{self.scores.low_zones}"
    
    def delete(self):
        """Deletes the climber from the leaderboard"""
        del self
    
class Leaderboard():
    def __init__(self):
        self.climbers = []

    def add_climber(self, *climbers):
        """Adds a climber to the leaderboard"""
        self.climbers.extend(climbers)
    
    def rank_climbers(self):
        """Sorts climbers by their scores. Key currently matches the USA Climbing Isolation scoring system."""
        def sort_key(climber):
            return (-climber.scores.tops.tops, -climber.scores.zones.zones, -climber.scores.low_zones.low_zones, 
            climber.scores.tops.attempts, climber.scores.zones.attempts, climber.scores.low_zones.attempts)
        self.climbers.sort(key=sort_key)
    
def create_climber(name, category, tops, zones, low_zones = (0, 0)):
    """Creates a climber object with the given name, category, and scores. Low zones are optional and scores are taken as tuples."""
    climber = Climber(name, category)
    climber.scores.tops = Scores.Top(*tops)
    climber.scores.zones = Scores.Zone(*zones)
    climber.scores.low_zones = Scores.LowZone(*low_zones)
    return climber

# tkinter window
class ScoringApp:
    def __init__(self, root):
        self.root = root
        self.root.title("Climbing Leaderboard")
        self.leaderboard = Leaderboard()
        
        # Main Frame
        self.main_frame = ttk.Frame(self.root)
        self.main_frame.grid(column= 0, row= 0, padx= 20, pady= 20, sticky= "nsew")
        self.main_frame.rowconfigure(0, weight= 1)
        self.main_frame.columnconfigure(0, weight= 1)
        self.main_frame.columnconfigure(1, weight= 1)

        # Resizing to fit the frame
        self.root.columnconfigure(0, weight= 1)
        self.root.rowconfigure(0, weight= 1)

        # Creating Left and Right sections
        self.create_left_frame()
        self.create_right_frame()

    def create_left_frame(self):
        """Create the left frame for inputs and buttons"""
        self.left_frame = ttk.Frame(self.main_frame)
        self.left_frame.grid(row=0, column=0, padx=10, sticky= 'nw')

        # Left Frame Resizing
        self.left_frame.columnconfigure(0, weight= 1)
        self.left_frame.columnconfigure(1, weight= 1)
        self.left_frame.rowconfigure(0, weight= 1)
        self.left_frame.rowconfigure(5, weight= 1)

        # Labels and Entry Fields
        labels = ["Climber Name", "Category", "Tops (count, attempts)", "Zones (count, attempts)", "Low Zones (count, attempts)"]
        self.entries = {}
        for i, label in enumerate(labels):
            ttk.Label(self.left_frame, text= label + ":").grid(row=i, column=0, padx=5, pady=5)
            entry = ttk.Entry(self.left_frame)
            entry.grid(row= i, column= 1, padx=5, pady=5, sticky= "w")
            self.entries[label] = entry #stores in dictionary for easy access
        
        # Leftside Buttons
        ttk.Button(self.left_frame, text="Add Climber", command=self.add_climber).grid(row= 5, column= 0, columnspan= 2, pady= 10)

        # Edit/Remove Climber Buttons
        add_remove_buttons = [
            ("Edit Climber", self.edit_climber),
            ("Remove Climber", self.remove_climber)
        ]
        for i, (text, command) in enumerate(add_remove_buttons):
            ttk.Button(self.left_frame, text= text, command= command).grid(row= 6, column= i, columnspan= 2, padx= 10, pady= 10, sticky= "w")

    def create_right_frame(self):
        """Create the right frame for the leaderboard display and associated buttons"""
        self.right_frame = ttk.Frame(self.main_frame)
        self.right_frame.grid(row=0, column=1, padx=10, sticky= 'nsew')

        #letting right frame expand to fill the space
        self.main_frame.columnconfigure(1, weight= 1)
        self.right_frame.rowconfigure(0, weight= 1)
        self.right_frame.columnconfigure(0, weight= 1)

        # Leaderboard Display
        self.leaderboard_text = tk.Text(self.right_frame, height=20, width=30)
        self.leaderboard_text.grid(row=0, column=0, columnspan= 2, pady=10, sticky= "nsew")
        self.disable_edits()

        # Rightside Buttons
        self.leaderboard_buttons = ttk.Frame(self.right_frame)
        self.leaderboard_buttons.grid(row=1, column=0, columnspan=2, pady=10)
        ttk.Button(self.leaderboard_buttons, text="Clear Leaderboard", command=self.clear_leaderboard_ask).pack(side= "left", padx= 10, pady= 10)
        ttk.Button(self.leaderboard_buttons, text="Show Leaderboard", command=self.show_leaderboard).pack(side= "left", padx= 10, pady= 10)

    # Functions and Logic
    def clear_entries(self):
        """Clears all entry fields"""
        self.enable_edits()
        for entry in self.entries.values():
            entry.delete(0, tk.END)
        self.disable_edits()

    def remove_climber(self):
        """Removes a climber from the Leaderboard"""
        try:
            name = self.entries["Climber Name"].get()
            found = False
            if name == "":
                raise ValueError("Climber Name cannot be empty")
            for climber in self.leaderboard.climbers[:]:
                if climber.name == name:
                    self.leaderboard.climbers.remove(climber)
                    climber.delete()
                    found = True
                    break
            if found:
                messagebox.showinfo("Success", f"{name} has been removed.")
                self.clear_entries()
                self.show_leaderboard()
            else:
                raise ValueError(f"Climber {name} not found in the leaderboard.")
        except Exception as e:
            messagebox.showerror("Error", str(e))

    def validate_scores(self, score_type):
        """Parses and validates the score from the entry field. Raises error an incorrect score is give"""
        scores = tuple(map(int, self.entries[score_type].get().split(',')))
        short_types = score_type.split()[0]
        if scores[0] > scores[1]:
            raise ValueError(f"{short_types}: '{scores[0]}' cannot be larger than attempts '{scores[1]}'")
        return scores

    def add_climber(self):
        """Adds a climber to the current leaderboard. Raises errors if the input is invalid"""
        try:
            name = self.entries["Climber Name"].get()
            if name == "":
                raise ValueError("Name cannot be empty")
            if name in [climber.name for climber in self.leaderboard.climbers]:
                raise ValueError(f"{name} already exists in the leaderboard")
            category = self.entries["Category"].get()
            tops = self.validate_scores("Tops (count, attempts)")
            zones = self.validate_scores("Zones (count, attempts)")
            low_zones = self.validate_scores("Low Zones (count, attempts)") if self.entries["Low Zones (count, attempts)"].get() else (0, 0)
   
            climber = create_climber(name, category, tops, zones, low_zones)
            self.leaderboard.add_climber(climber)
            messagebox.showinfo("Success", f"{name} added to the leaderboard!")
            self.clear_entries()
            self.show_leaderboard()
        except Exception as e:
            messagebox.showerror("Error", f"Invalid input: {e}")

    def show_leaderboard(self):
        """Displays the current leaderboard, with rankings"""
        self.leaderboard.rank_climbers()
        self.enable_edits()
        self.leaderboard_text.delete(1.0, tk.END)
        for climber in self.leaderboard.climbers:
            self.leaderboard_text.insert(tk.END, f"{climber}\n\n")
        self.disable_edits()

    def edit_climber(self):
        """Edits the score of an existing climber.
            If a new value is entered in the entry field, it is validated and updated"""
        try: 
            name = self.entries["Climber Name"].get()  
            if not name:
                raise ValueError("Climber Name cannot be empty")  
             
            climber = next((c for c in self.leaderboard.climbers if c.name == name), None)
            if not climber:
                raise ValueError(f"Climber {name} not found.")
            
            category = self.entries["Category"].get()
            if category:
                climber.category = category
            
            if self.entries["Tops (count, attempts)"].get():
                tops = self.validate_scores("Tops (count, attempts)")
                climber.scores.tops = Scores.Top(*tops)

            if self.entries["Zones (count, attempts)"].get():
                zones = self.validate_scores("Zones (count, attempts)")
                climber.scores.zones = Scores.Zone(*zones)
        
            if self.entries["Low Zones (count, attempts)"].get():
                low_zones = self.validate_scores("Low Zones (count, attempts)")
                climber.scores.low_zones = Scores.LowZone(*low_zones)

            messagebox.showinfo("Success", f"Climber '{name}' has been updated.")
            self.clear_entries()
            self.show_leaderboard()
        except Exception as e:
            messagebox.showerror("Error", f"Error updating climber '{name}':\n{str(e)}")

    def clear_leaderboard_ask(self):
        """Asks for confirmation before clearing the entries"""
        response = messagebox.askyesno("Clear Leaderboard", "Are you sure you want to clear the leaderboard?")
        if response:
            self.clear_leaderboard()
        
    def clear_leaderboard(self):
        """Clears the leaderboard values"""
        self.leaderboard.climbers = []
        self.enable_edits()
        self.leaderboard_text.delete(1.0, tk.END)
        messagebox.showinfo("Success", "Leaderboard has been cleared.")
        self.disable_edits()

    def enable_edits(self):
        self.leaderboard_text.config(state= tk.NORMAL)

    def disable_edits(self):
        self.leaderboard_text.config(state= tk.DISABLED)

if __name__ == "__main__":
    root = tk.Tk()
    app = ScoringApp(root)
    root.mainloop()

In [ ]:
# old layout that still works but isn't OOP
# tkinter window
import tkinter as tk
from tkinter import ttk, messagebox

class ScoringApp:
    def __init__(self, root):
        self.root = root
        self.root.title("Climbing Leaderboard")
        self.leaderboard = Leaderboard()
        
        # Main Frame
        self.main_frame = ttk.Frame(self.root)
        self.main_frame.grid(column= 0, row= 0, padx= 20, pady= 20, sticky= "nsew")

        # Resizing to fit the frame
        self.root.columnconfigure(0, weight= 1)
        self.root.columnconfigure(1, weight= 1)
        self.root.rowconfigure(0, weight= 1)

        # Creating Left and Right sections
        self.create_left_frame()
        self.create_right_frame()

    def create_left_frame(self):
        """Create the left frame for inputs and buttons"""
        self.left_frame = ttk.Frame(self.main_frame)
        self.left_frame.grid(row=0, column=0, padx=10, sticky= 'nw')

        # Labels and Entry Fields
        labels = ["Climber Name", "Category", "Tops (count, attempts)", "Zones (count, attempts)", "Low Zones (count, attempts)"]
        self.entries = {}
        for i, label in enumerate(labels):
            ttk.Label(self.left_frame, text= label + ":").grid(row=i, column=0, padx=5, pady=5)
            entry = ttk.Entry(self.left_frame)
            entry.grid(row= i, column= 1, padx=5, pady=5, sticky= "w")
            self.entries[label] = entry #stores in dictionary for easy access

            # Climber Name
        ttk.Label(self.left_frame, text="Climber Name:").grid(row=0, column=0, padx=5, pady=5)
        self.name_entry = ttk.Entry(self.left_frame)
        self.name_entry.grid(row=0, column=1, padx=5, pady=5, sticky= "w")

            # Climber Category
        ttk.Label(self.left_frame, text="Category:").grid(row=1, column=0, padx=5, pady=5)
        self.category_entry = ttk.Entry(self.left_frame)
        self.category_entry.grid(row=1, column= 1, padx=5, pady=5, sticky= "w")
        
            # Climber Scores, Tops, Zones, Low Zones
        ttk.Label(self.left_frame, text="Tops (count, attempts):").grid(row=2, column=0, padx=5, pady=5)
        self.tops_entry = ttk.Entry(self.left_frame)
        self.tops_entry.grid(row=2, column= 1, padx=5, pady=5, sticky= "w")
        
        ttk.Label(self.left_frame, text="Zones (count, attempts):").grid(row=3, column=0, padx=5, pady=5)
        self.zones_entry = ttk.Entry(self.left_frame)
        self.zones_entry.grid(row=3, column=1, padx=5, pady=5, sticky= "w")
        
        ttk.Label(self.left_frame, text="Low Zones (count, attempts):").grid(row=4, column=0, padx=5, pady=5)
        self.low_zones_entry = ttk.Entry(self.left_frame)
        self.low_zones_entry.grid(row=4, column=1, padx=5, pady=5, sticky= "w")

        
        # Leftside Buttons
        ttk.Button(self.left_frame, text="Add Climber", command=self.add_climber).grid(row= 5, column= 0, columnspan= 2, pady= 10)
        ttk.Button(self.left_frame, text="Edit Climber").grid(row= 6, column= 0, columnspan= 2, padx= 10, pady= 10, sticky= "w")
        ttk.Button(self.left_frame, text= "Remove Climber").grid(row= 6, column= 1, columnspan= 2, padx= 10, pady= 10, sticky= "e")

    def create_right_frame(self):
        """Create the right frame for the leaderboard display and associated buttons"""
        self.right_frame = ttk.Frame(self.main_frame)
        self.right_frame.grid(row=0, column=1, padx=10, sticky= 'ne')

        # Leaderboard Display
        self.leaderboard_text = tk.Text(self.right_frame, height=20, width=50)
        self.leaderboard_text.grid(row=0, column=0, columnspan= 2, pady=10, sticky= "nsew")

        # Rightside Buttons
        ttk.Button(self.right_frame, text="Clear Leaderboard", command=self.clear_leaderboard_ask).grid(row= 1, column= 0, columnspan= 2, pady= 10)
        ttk.Button(self.right_frame, text="Show Leaderboard", command=self.show_leaderboard).grid(row= 1, column= 1, columnspan= 2, pady= 10)

    # Functions and Logic
    def clear_entries(self):
        """Clears all but category entry fields"""
        self.name_entry.delete(0, tk.END)
        self.tops_entry.delete(0, tk.END)
        self.zones_entry.delete(0, tk.END)
        self.low_zones_entry.delete(0, tk.END)

    def add_climber(self):
        """Adds a climber to the current leaderboard"""
        try:
            name = self.name_entry.get()
            category = self.category_entry.get()
            tops = tuple(map(int, self.entries["Tops (count, attempts)"].get().split(',')))
            zones = tuple(map(int, self.entries["Zones (count, attempts)"].get().split(',')))
            low_zones = tuple(map(int, self.entries["Low Zones (count, attempts)"].get().split(',')) if self.entries["Low Zones (count, attempts)"].get() else (0, 0))

            
            climber = create_climber(name, category, tops, zones, low_zones)
            self.leaderboard.add_climber(climber)
            messagebox.showinfo("Success", f"{name} added to the leaderboard!")
            self.clear_entries()
        except Exception as e:
            messagebox.showerror("Error", f"Invalid input: {e}")

    def show_leaderboard(self):
        """Displays the current leaderboard, with rankings"""
        self.leaderboard.rank_climbers()
        self.leaderboard_text.delete(1.0, tk.END)
        for climber in self.leaderboard.climbers:
            self.leaderboard_text.insert(tk.END, f"{climber}\n\n")

    def edit_climber(self):
        """Edits the score of an existing climber"""
        try: 
            name = self.name_entry.get()
            category = self.category_entry.get()
            tops = tuple(map(int, self.tops_entry.get().split(',')))
            zones = tuple(map(int, self.zones_entry.get().split(',')))
            low_zones = tuple(map(int, self.low_zones_entry.get().split(','))) if self.low_zones_entry.get() else (0, 0)

            for climber in self.leaderboard.climbers:
                if climber.name == name:
                    climber.scores.tops = Scores.Top(*tops)
                    climber.scores.zones = Scores.Zone(*zones)
                    climber.scores.low_zones = Scores.LowZone(*low_zones)
                    messagebox.showinfo("Success", f"{name} has been updated.")
                    self.clear_entries()
                    return
        except Exception as e:
            messagebox.showerror("Error", f"invalid input {e}")

    def clear_leaderboard_ask(self):
        """Asks for confirmation before clearing the entries"""
        response = messagebox.askyesno("Clear Leaderboard", "Are you sure you want to clear the leaderboard?")
        if response:
            self.clear_leaderboard()
        
    def clear_leaderboard(self):
        """Clears the leaderboard values"""
        self.leaderboard.climbers = []
        self.leaderboard_text.delete(1.0, tk.END)
        messagebox.showinfo("Success", "Leaderboard has been cleared.")

if __name__ == "__main__":
    root = tk.Tk()
    app = ScoringApp(root)
    root.mainloop()


In [45]:
# Leaderboard to the right
class LeaderboardApp:
    def __init__(self, root):
        self.root = root
        self.root.title("Climbing Leaderboard")
        
        # Create Main Frame to Hold Everything
        self.main_frame = ttk.Frame(self.root)
        self.main_frame.pack(expand=True, fill="both", padx=20, pady=20)

        # Create Left and Right Sections
        self.create_left_frame()
        self.create_right_frame()

    def create_left_frame(self):
        """Create the left frame for inputs and buttons."""
        self.left_frame = ttk.Frame(self.main_frame)
        self.left_frame.grid(row=0, column=0, padx=10, sticky="nw")

        labels = ["Climber Name:", "Tops (count, attempts):", 
                  "Zones (count, attempts):", "Low Zones (count, attempts):"]
        
        self.entries = {}
        for i, label in enumerate(labels):
            ttk.Label(self.left_frame, text=label).grid(row=i, column=0, padx=5, pady=5, sticky="e")
            entry = ttk.Entry(self.left_frame, width=30)
            entry.grid(row=i, column=1, padx=5, pady=5, sticky="w")
            self.entries[label] = entry  # Store in dictionary for easy access

        # Button to Add Climber (Stays on Left Side)
        ttk.Button(self.left_frame, text="Add Climber").grid(row=len(labels), column=0, columnspan=2, pady=10)

        # Bottom Buttons (Only "Edit" and "Remove" stay on left)
        self.bottom_frame = ttk.Frame(self.left_frame)
        self.bottom_frame.grid(row=len(labels)+1, column=0, columnspan=2, pady=10)

        ttk.Button(self.bottom_frame, text="Edit Climber").grid(row=0, column=0, padx=5)
        ttk.Button(self.bottom_frame, text="Remove Climber").grid(row=0, column=1, padx=5)

    def create_right_frame(self):
        """Create the right frame for the leaderboard display and leaderboard-related buttons."""
        self.right_frame = ttk.Frame(self.main_frame)
        self.right_frame.grid(row=0, column=1, padx=10, sticky="n")

        # Leaderboard Text Box
        self.leaderboard_text = tk.Text(self.right_frame, height=20, width=50)
        self.leaderboard_text.pack(expand=True, fill="both")

        # Buttons Below the Leaderboard
        self.leaderboard_buttons_frame = ttk.Frame(self.right_frame)
        self.leaderboard_buttons_frame.pack(pady=10)

        ttk.Button(self.leaderboard_buttons_frame, text="Show Leaderboard").pack(side="left", padx=5)
        ttk.Button(self.leaderboard_buttons_frame, text="Clear Leaderboard").pack(side="left", padx=5)

if __name__ == "__main__":
    root = tk.Tk()
    app = LeaderboardApp(root)
    root.mainloop()

In [42]:
# Leaderboard below 
class LeaderboardApp:
    def __init__(self, root):
        self.root = root
        self.root.title("Climbing Leaderboard")
        
        # Configure root layout for expansion
        self.root.columnconfigure(0, weight=1)
        self.root.rowconfigure(2, weight=1)  # Make the leaderboard expand

        # Create Frames
        self.create_input_frame()
        self.create_button_frame()
        self.create_leaderboard()
        self.create_bottom_buttons()

    def create_input_frame(self):
        """Create the input fields for climber details."""
        self.input_frame = ttk.Frame(self.root)
        self.input_frame.grid(row=0, column=0, padx=10, pady=10, sticky="ew")

        labels = ["Climber Name:", "Category:", "Tops (count, attempts):", 
                  "Zones (count, attempts):", "Low Zones (count, attempts):"]
        
        self.entries = {}
        for i, label in enumerate(labels):
            ttk.Label(self.input_frame, text=label).grid(row=i, column=0, padx=5, pady=5, sticky="e")
            entry = ttk.Entry(self.input_frame, width=30)
            entry.grid(row=i, column=1, padx=5, pady=5, sticky="w")
            self.entries[label] = entry  # Store in dictionary for easy access

    def create_button_frame(self):
        """Create buttons for adding and showing leaderboard."""
        self.button_frame = ttk.Frame(self.root)
        self.button_frame.grid(row=1, column=0, pady=10)

        ttk.Button(self.button_frame, text="Add Climber").grid(row=0, column=0, padx=5)
        ttk.Button(self.button_frame, text="Show Leaderboard").grid(row=0, column=1, padx=5)

    def create_leaderboard(self):
        """Create the text box for displaying the leaderboard."""
        self.leaderboard_text = tk.Text(self.root, height=15, width=50)
        self.leaderboard_text.grid(row=2, column=0, pady=10, padx=10, sticky="nsew")

    def create_bottom_buttons(self):
        """Create edit and clear buttons at the bottom."""
        self.bottom_frame = ttk.Frame(self.root)
        self.bottom_frame.grid(row=3, column=0, pady=10)

        ttk.Button(self.bottom_frame, text="Edit Climber").grid(row=0, column=0, padx=5)
        ttk.Button(self.bottom_frame, text="Clear Leaderboard").grid(row=0, column=1, padx=5)
        ttk.Button(self.bottom_frame, text="Remove Climber").grid(row=0, column=2, padx=5)

if __name__ == "__main__":
    root = tk.Tk()
    app = LeaderboardApp(root)
    root.mainloop()